# IMPORT LIBRARIES

In [6]:
import json
import re
import numpy as np
from typing import List, Dict, Any, Optional, Tuple, Set
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from neo4j import GraphDatabase
import time
from collections import defaultdict

# CONFIGURATION


In [ ]:
# Neo4j Configuration
NEO4J_URI = "neo4j://127.0.0.1:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

# Pipeline Configuration
MAX_NODES = 1500
TOP_K = 5
SIM_THRESHOLD = 0.6
EXACT_MATCH_THRESHOLD = 0.9
LLM_MODEL = "meta-llama/llama-4-scout-17b-16e-instruct"
MAX_PATH_LENGTH = 4  # Maximum relationship hops to search
MAX_PATHS_PER_PAIR = 5  # Increased for better pruning
TOP_PATHS_FOR_PRUNING = 3  # Top paths to consider for LLM pruning

# Medical Entity types
ENTITY_TYPES = [
    "disease",
    "drug"
]
# Removed "Embedding" as it might be a category mistake

# Entity Property Schemas
# Entity Property Schemas adapted to biomedical entity types
ENTITY_PROPERTY_SCHEMAS = {
    "disease": {
        "properties": [
            "SNOMEDCT_US_definition",
            "mayo_causes",
            "mayo_complications",
            "mayo_prevention",
            "mayo_risk_factors",
            "mayo_see_doc",
            "mayo_symptoms",
            "mondo_definitions",
            "node_id",
            "node_index",
            "node_name",
            "node_source",
            "orphanet_clinical_description",
            "orphanet_definition",
            "orphanet_epidemiology",
            "orphanet_management_and_treatment",
            "orphanet_prevalence",
            "umls_descriptions"
        ],
        "primary_keys": ["node_id", "node_name"],
        "display_properties": [
            "node_name",
            "mondo_definitions",
            "mayo_symptoms",
            "orphanet_prevalence"
        ]
    },
    "drug": {
        "properties": [
            "atc_4",
            "category",
            "clogp",
            "description",
            "group",
            "half_life",
            "indication",
            "mechanism_of_action",
            "molecular_weight",
            "node_id",
            "node_index",
            "node_name",
            "node_source",
            "pathway",
            "pharmacodynamics",
            "protein_binding",
            "state",
            "tpsa"
        ],
        "primary_keys": ["node_id", "node_name"],
        "display_properties": [
            "node_name",
            "description",
            "indication",
            "mechanism_of_action",
            "category"
        ]
    }
}

# REVISED Property weightings for similarity
PROPERTY_WEIGHTS = {
    "exact_primary_key": 10.0,      # Very high for exact primary key matches
    "exact_display_property": 5.0,   # High for exact display property matches
    "contains_primary_key": 3.0,     # Good for contains matches
    "contains_display_property": 2.0, # Medium for contains matches
    "other_property_match": 1.0,     # Low for other properties
    "name_exact": 8.0,               # High for exact name matches
    "name_contains": 4.0             # Medium for name contains
}

# Entity examples for better prompting
# Entity examples for biomedical entity types
ENTITY_EXAMPLES = {
    "disease": [
        "folliculotropic mycosis fungoides",
        "localized pagetoid reticulosis", 
        "erythema multiforme",
        "classic Hodgkin lymphoma, lymphocyte-rich type",
        "Hodgkin's paragranuloma",
        "lymphosarcoma",
        "hairy cell leukemia variant"
    ],
    "drug": [
        "Diethylstilbestrol",
        "Liothyronine",
        "Levothyroxine",
        "Hydrocortisone cypionate",
        "Hydrocortisone phosphate",
        "Hydrocortisone probutate", 
        "Hydrocortisone valerate",
        "Prednisolone phosphate",
        "Prednisolone acetate",
        "Betamethasone phosphate"
    ]
}

# NEO4J CONNECTION & KG LOADING


In [8]:
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USER, NEO4J_PASSWORD)
)

# Global variables to store statistics
label_counts_global = defaultdict(int)
type_counts_global = defaultdict(int)

def load_kg_nodes_with_properties() -> List[Dict]:
    """Load ONLY disease and drug nodes from the KG and combine all properties for embeddings."""
    global label_counts_global, type_counts_global
    
    with driver.session() as s:
        # Count disease and drug nodes
        count_result = s.run("""
        MATCH (n)
        WHERE any(label IN labels(n) WHERE label IN ['disease', 'drug'])
        RETURN count(n) AS total_disease_drug_nodes
        """).data()
        
        total_disease_drug_nodes = count_result[0]["total_disease_drug_nodes"]
        print(f"📊 Total disease and drug nodes in KG: {total_disease_drug_nodes}")
        
        # Load ONLY disease and drug nodes with properties
        nodes = s.run("""
        MATCH (n)
        WHERE any(label IN labels(n) WHERE label IN ['disease', 'drug'])
        RETURN elementId(n) AS node_id, labels(n) AS labels, properties(n) AS properties
        """).data()
    
    print(f"📥 Loaded {len(nodes)} disease/drug nodes from Neo4j query")
    
    enhanced_nodes = []
    label_counts_local = defaultdict(int)
    type_counts_local = defaultdict(int)
    
    for node in nodes:
        props = dict(node["properties"])
        labels = node["labels"]
        
        # Count label frequencies
        for label in labels:
            label_counts_local[label] += 1
        
        # Create combined text from ALL property values
        combined_text = " ".join(str(v) for v in props.values() if v is not None)
        
        # Determine best node name - prioritize node_name property
        node_name = None
        
        # First, check for node_name property (most reliable)
        if "node_name" in props and props["node_name"]:
            node_name = str(props["node_name"])
        else:
            # Try other name candidates
            name_candidates = [
                "name", "Name", "NAME",
                "description", "Description",
                "SNOMEDCT_US_definition", "mondo_definitions",
                "indication", "mechanism_of_action",
                "title", "Title", "label", "Label"
            ]
            
            for prop in name_candidates:
                if prop in props and props[prop]:
                    node_name = str(props[prop])
                    # Take only first 100 chars to avoid huge text
                    if len(node_name) > 100:
                        node_name = node_name[:100] + "..."
                    break
        
        # Fallback: use first property value
        if not node_name and props:
            for value in props.values():
                if value is not None:
                    node_name = str(value)
                    if len(node_name) > 100:
                        node_name = node_name[:100] + "..."
                    break
        
        # Final fallback
        if not node_name:
            node_name = f"Node_{len(enhanced_nodes)+1}"
        
        # Determine entity type based on labels
        entity_type = "Unknown"
        
        # Check if node has disease or drug label
        for label in labels:
            if label.lower() in ['disease', 'drug']:
                entity_type = label.lower()
                break
        
        # If still unknown, try to infer from properties
        if entity_type == "Unknown":
            # Check for disease-specific properties
            disease_props = ["SNOMEDCT_US_definition", "mondo_definitions", "mayo_symptoms", "orphanet_definition"]
            if any(prop in props for prop in disease_props):
                entity_type = "disease"
            # Check for drug-specific properties
            elif any(prop in props for prop in ["mechanism_of_action", "atc_4", "indication", "pharmacodynamics"]):
                entity_type = "drug"
            else:
                # Fuzzy label matching
                label_str = " ".join(labels).lower()
                if "disease" in label_str or "disorder" in label_str or "syndrome" in label_str:
                    entity_type = "disease"
                elif "drug" in label_str or "compound" in label_str or "medication" in label_str:
                    entity_type = "drug"
                else:
                    # Default to first label
                    entity_type = labels[0] if labels else "Unknown"
        
        # Skip if not disease or drug (shouldn't happen with our query, but just in case)
        if entity_type not in ["disease", "drug"]:
            continue
        
        # Count entity types
        type_counts_local[entity_type] += 1
        
        # Create enhanced property text for better embeddings
        property_text = node_name
        
        # Use schema-based property enhancement
        if entity_type in ENTITY_PROPERTY_SCHEMAS:
            schema = ENTITY_PROPERTY_SCHEMAS[entity_type]
            
            # Add primary keys to property text
            for prop in schema.get("primary_keys", []):
                if prop in props and props[prop]:
                    prop_value = str(props[prop])
                    if prop_value and prop_value.lower() not in property_text.lower():
                        property_text += f" {prop_value}"
            
            # Add display properties to property text
            for prop in schema.get("display_properties", []):
                if prop in props and props[prop]:
                    prop_value = str(props[prop])
                    if prop_value and prop_value.lower() not in property_text.lower():
                        property_text += f" {prop_value}"
        else:
            # For unknown schema, add important properties
            important_props = ["description", "definition", "indication", "symptoms", "mechanism"]
            added_count = 0
            for prop in important_props:
                if prop in props and props[prop] and added_count < 3:
                    prop_value = str(props[prop])
                    if prop_value and prop_value.lower() not in property_text.lower():
                        property_text += f" {prop_value}"
                        added_count += 1
        
        # Limit property text length to avoid huge embeddings
        if len(property_text) > 500:
            property_text = property_text[:500] + "..."
        
        enhanced_nodes.append({
            "node_id": str(node["node_id"]),
            "node_name": node_name,
            "labels": labels,
            "entity_type": entity_type,
            "properties": props,
            "property_keys": list(props.keys()),
            "combined_text": combined_text.strip(),
            "property_text": property_text.strip(),
            "text_length": len(property_text.strip())
        })
    
    # Update global statistics
    for label, count in label_counts_local.items():
        label_counts_global[label] += count
    
    for entity_type, count in type_counts_local.items():
        type_counts_global[entity_type] += count
    
    # Print label distribution
    print(f"\n📊 Label Distribution (disease/drug):")
    for label, count in sorted(label_counts_local.items(), key=lambda x: x[1], reverse=True):
        print(f"   {label}: {count}")
    
    # Print entity type distribution
    print(f"\n📊 Entity Type Distribution:")
    
    # Count and display disease vs drug distribution
    disease_count = type_counts_local.get("disease", 0)
    drug_count = type_counts_local.get("drug", 0)
    total_filtered = disease_count + drug_count
    
    if total_filtered > 0:
        print(f"   Disease: {disease_count} ({disease_count/total_filtered*100:.1f}%)")
        print(f"   Drug: {drug_count} ({drug_count/total_filtered*100:.1f}%)")
    
    # Print other entity types if any
    for entity_type, count in sorted(type_counts_local.items(), key=lambda x: x[1], reverse=True):
        if entity_type not in ["disease", "drug"]:
            print(f"   {entity_type}: {count}")
    
    # Print text length statistics
    if enhanced_nodes:
        avg_length = sum(node["text_length"] for node in enhanced_nodes) / len(enhanced_nodes)
        print(f"\n📏 Average property text length: {avg_length:.0f} characters")
    
    return enhanced_nodes

# Load KG nodes
print("🔄 Loading disease and drug nodes from knowledge graph...")
KG_NODES = load_kg_nodes_with_properties()
print(f"✅ Successfully loaded {len(KG_NODES)} disease/drug nodes from knowledge graph")

if len(KG_NODES) == 0:
    print("❌ ERROR: No disease or drug nodes loaded!")
    print("   Possible reasons:")
    print("   1. Check Neo4j connection and credentials")
    print("   2. Verify nodes have 'disease' or 'drug' labels")
    print("   3. Check if database contains disease/drug data")
    exit(1)

# Use both combined_text and property_text for different purposes
KG_TEXTS = [node["property_text"] for node in KG_NODES]
print(f"\n📋 Sample disease/drug nodes loaded:")
for i, node in enumerate(KG_NODES[:5]):
    entity_type = node.get('entity_type', 'Unknown')
    props_count = len(node['properties'])
    name_preview = node['node_name'][:60] + "..." if len(node['node_name']) > 60 else node['node_name']
    text_preview = node['property_text'][:80] + "..." if len(node['property_text']) > 80 else node['property_text']
    print(f"  {i+1}. {name_preview} ({entity_type}) - Props: {props_count}")
    print(f"     Preview: {text_preview}")

# Optional: Save summary to file - using the GLOBAL variables
import json
with open("disease_drug_nodes_summary.json", "w") as f:
    summary = {
        "total_nodes": len(KG_NODES),
        "entity_type_counts": dict(type_counts_global),
        "label_counts": dict(label_counts_global),
        "sample_nodes": KG_NODES[:10]
    }
    json.dump(summary, f, indent=2, default=str)
print(f"\n💾 Summary saved to disease_drug_nodes_summary.json")

🔄 Loading disease and drug nodes from knowledge graph...
📊 Total disease and drug nodes in KG: 25037
📥 Loaded 25037 disease/drug nodes from Neo4j query

📊 Label Distribution (disease/drug):
   disease: 17080
   drug: 7957

📊 Entity Type Distribution:
   Disease: 17080 (68.2%)
   Drug: 7957 (31.8%)

📏 Average property text length: 268 characters
✅ Successfully loaded 25037 disease/drug nodes from knowledge graph

📋 Sample disease/drug nodes loaded:
  1. drug-induced dyskinesia (disease) - Props: 14
     Preview: drug-induced dyskinesia 6732 ['Abnormal movements, including hyperkinesis; hypok...
  2. hyperemesis gravidarum (disease) (disease) - Props: 6
     Preview: hyperemesis gravidarum (disease) 6791 ['Severe, intractable vomiting during preg...
  3. torsades de pointes (disease) - Props: 12
     Preview: torsades de pointes 5478 ['A malignant form of polymorphic ventricular tachycard...
  4. focal hand dystonia (disease) - Props: 15
     Preview: focal hand dystonia 482 ['A focal dy

# ENHANCED EMBEDDINGS & SIMILARITY


In [9]:
print("\n🔄 Loading embedding model...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")
print(f"🔄 Encoding {len(KG_TEXTS)} KG nodes with enhanced property text...")

# Batch encoding to handle memory better
try:
    KG_EMBEDDINGS = embedder.encode(KG_TEXTS, show_progress_bar=True, convert_to_numpy=True)
    print(f"✅ KG embeddings computed for {len(KG_EMBEDDINGS)} nodes")
except Exception as e:
    print(f"❌ Error encoding nodes: {e}")
    print("🔄 Trying with smaller batch size...")
    # Encode in batches
    batch_size = 100
    embeddings_list = []
    for i in range(0, len(KG_TEXTS), batch_size):
        batch = KG_TEXTS[i:i+batch_size]
        batch_embeddings = embedder.encode(batch, show_progress_bar=False, convert_to_numpy=True)
        embeddings_list.append(batch_embeddings)
        print(f"  Encoded batch {i//batch_size + 1}/{(len(KG_TEXTS)+batch_size-1)//batch_size}")
    
    KG_EMBEDDINGS = np.vstack(embeddings_list)
    print(f"✅ KG embeddings computed in batches for {len(KG_EMBEDDINGS)} nodes")

def safe_json_parse(text: str) -> Dict:
    """Safely parse JSON from LLM response."""
    if not text:
        return {}
    
    # Remove code blocks
    text = re.sub(r"```json\s*|```\s*", "", text).strip()
    
    # Try to find and fix common JSON issues
    # Fix missing quotes around keys
    text = re.sub(r'(\w+)\s*:', r'"\1":', text)
    
    # Fix trailing commas
    text = re.sub(r',\s*}', '}', text)
    text = re.sub(r',\s*]', ']', text)
    
    # Try to parse as JSON
    try:
        parsed = json.loads(text)
        return parsed
    except json.JSONDecodeError as e:
        # Try to find JSON object/array
        json_pattern = r'(\{.*\}|\[.*\])'
        matches = re.findall(json_pattern, text, re.DOTALL)
        
        if matches:
            try:
                return json.loads(matches[0])
            except:
                # Last attempt: try eval for simple dicts (be careful!)
                try:
                    # Only use eval for trusted content in development
                    if isinstance(eval(matches[0]), (dict, list)):
                        return eval(matches[0])
                except:
                    return {}
    
    return {}


🔄 Loading embedding model...
🔄 Encoding 25037 KG nodes with enhanced property text...


Batches:   0%|          | 0/783 [00:00<?, ?it/s]

✅ KG embeddings computed for 25037 nodes


# ENHANCED ENTITY EXTRACTION


In [10]:
def extract_entities_from_text(text: str, client) -> List[Dict]:
    """
    Extract fertility-related biomedical entities from text using LLM.
    Focuses on fertility diseases and their corresponding drugs/treatments.
    """
    
    # Get fertility-specific examples
    fertility_diseases = ENTITY_EXAMPLES.get("disease", [])
    fertility_drugs = ENTITY_EXAMPLES.get("drug", [])
    
    extraction_prompt = f"""
    TASK: Extract ONLY fertility-related biomedical entities from the text below.
    Focus on reproductive health diseases and their corresponding drugs/treatments.

    TEXT: "{text}"

    FERTILITY ENTITY TYPES:
    1. DISEASE: Fertility/reproductive health conditions ONLY
       Examples: {', '.join(fertility_diseases[:5])}
       Types: Infertility conditions, reproductive disorders, hormonal imbalances affecting fertility
    
    2. DRUG: Fertility medications and treatments ONLY  
       Examples: {', '.join(fertility_drugs[:5])}
       Types: Fertility drugs, hormone therapies, assisted reproduction medications

    STRICT RULES:
    1. Extract ONLY fertility-related entities
    2. Skip general diseases/drugs not related to reproduction
    3. Extract EXACT text spans as they appear
    4. Include confidence based on fertility relevance (0.0 to 1.0)
    5. Return valid JSON only

    EXAMPLES OF FERTILITY ENTITIES (EXTRACT):
    - "Polycystic ovary syndrome (PCOS)" → disease
    - "Clomiphene citrate" → drug  
    - "Endometriosis" → disease
    - "In vitro fertilization (IVF)" → drug/treatment
    - "Premature ovarian insufficiency" → disease
    - "Gonadotropins" → drug

    EXAMPLES OF NON-FERTILITY (IGNORE):
    - "Hypertension" → IGNORE (unless in fertility context)
    - "Aspirin" → IGNORE (unless prescribed for fertility)
    - "Diabetes" → IGNORE (unless causing infertility)
    - "Common cold" → IGNORE

    CONTEXT-BASED EXTRACTION:
    If text mentions "infertility", "fertility treatment", "reproduction", "pregnancy" etc.,
    then extract related diseases/drugs even if not explicitly fertility-related.

    EXAMPLES:
    Input: "Patient with Polycystic ovary syndrome was treated with Clomiphene citrate"
    Output: {{"Entity": [
      {{"id": "1", "type": "disease", "name": "Polycystic ovary syndrome", "confidence": 0.95}},
      {{"id": "2", "type": "drug", "name": "Clomiphene citrate", "confidence": 0.9}}
    ]}}

    Input: "Endometriosis causing infertility treated with GnRH agonists"
    Output: {{"Entity": [
      {{"id": "1", "type": "disease", "name": "Endometriosis", "confidence": 0.9}},
      {{"id": "2", "type": "drug", "name": "GnRH agonists", "confidence": 0.85}}
    ]}}

    Input: "Diethylstilbestrol exposure linked to fertility issues"
    Output: {{"Entity": [
      {{"id": "1", "type": "drug", "name": "Diethylstilbestrol", "confidence": 0.95}},
      {{"id": "2", "type": "disease", "name": "fertility issues", "confidence": 0.8}}
    ]}}

    OUTPUT FORMAT:
    {{
      "Entity": [
        {{"id": "1", "type": "disease" OR "drug", "name": "exact_name_from_text", "confidence": 0.95}}
      ]
    }}
    """
    
    try:
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "You are a fertility medicine specialist. Extract ONLY fertility-related diseases and drugs. Ignore unrelated medical entities."},
                {"role": "user", "content": extraction_prompt}
            ],
            temperature=0.1,
            max_tokens=800
        )
        
        content = response.choices[0].message.content
        parsed = safe_json_parse(content)
        
        # Handle both "Entity" and "entities" keys
        entities = []
        if "Entity" in parsed and isinstance(parsed["Entity"], list):
            entities = parsed["Entity"]
        elif "entities" in parsed and isinstance(parsed["entities"], list):
            entities = parsed["entities"]
        else:
            print(f"⚠️ LLM extraction failed, using fertility-specific fallback")
            return fertility_fallback_extraction(text)
        
        # Filter to only fertility-related entities
        fertility_entities = []
        fertility_keywords = [
            "fertility", "infertility", "reproductive", "ovarian", "uterine", 
            "endometri", "pcos", "sperm", "oocyte", "embryo", "ivf", 
            "pregnancy", "conception", "menstrual", "hormon", "gonad"
        ]
        
        for entity in entities:
            if not isinstance(entity, dict) or "name" not in entity or "type" not in entity:
                continue
                
            entity_name = entity["name"].lower()
            entity_type = entity["type"].lower()
            
            # Check if entity is fertility-related
            is_fertility_related = False
            
            # Check entity name for fertility keywords
            if any(keyword in entity_name for keyword in fertility_keywords):
                is_fertility_related = True
            # Check if it's in our fertility examples
            elif entity["name"] in fertility_diseases or entity["name"] in fertility_drugs:
                is_fertility_related = True
            # Check if it's a known fertility condition/drug pattern
            elif entity_type == "disease" and any(term in entity_name for term in 
                  ["infertility", "endometriosis", "pcos", "fibroid", "amenorrhea"]):
                is_fertility_related = True
            elif entity_type == "drug" and any(term in entity_name for term in 
                  ["clomiphene", "letrozole", "gonadotropin", "fsh", "lh", "hcg"]):
                is_fertility_related = True
            
            if is_fertility_related:
                # Ensure proper entity type
                if entity_type not in ["disease", "drug"]:
                    # Map to correct type based on content
                    if any(term in entity_name for term in ["syndrome", "disease", "disorder", "infertility"]):
                        entity["type"] = "disease"
                    else:
                        entity["type"] = "drug"
                
                # Add confidence if missing
                if "confidence" not in entity:
                    # Adjust confidence based on fertility relevance
                    if any(keyword in entity_name for keyword in fertility_keywords):
                        entity["confidence"] = 0.85
                    else:
                        entity["confidence"] = 0.7
                
                # Add id if missing
                if "id" not in entity:
                    entity["id"] = str(len(fertility_entities) + 1)
                
                fertility_entities.append(entity)
        
        # If no fertility entities found but text has fertility context, use fallback
        if not fertility_entities and any(keyword in text.lower() for keyword in fertility_keywords):
            return fertility_fallback_extraction(text)
        
        return fertility_entities
            
    except Exception as e:
        print(f"❌ Error in fertility entity extraction: {e}")
        return fertility_fallback_extraction(text)

def fertility_fallback_extraction(text: str) -> List[Dict]:
    """Fallback method specifically for fertility-related entities."""
    entities = []
    
    # Fertility-specific patterns
    fertility_patterns = {
        "disease": [
            # Infertility conditions
            r'\b(?:infertility|subfertility)\b',
            r'\b(?:male|female) factor (?:infertility|subfertility)\b',
            r'\b(?:unexplained|idiopathic) infertility\b',
            
            # Ovarian disorders
            r'\b(?:Polycystic ovary syndrome|PCOS|polycystic ovarian syndrome)\b',
            r'\b(?:Premature ovarian (?:insufficiency|failure)|POI|POF)\b',
            r'\b(?:Diminished|Decreased) ovarian reserve\b',
            r'\bPoor ovarian response\b',
            r'\bAnovulation\b',
            r'\bLuteal phase defect\b',
            
            # Uterine disorders
            r'\bEndometriosis\b',
            r'\b(?:Uterine|Uterus) fibroids?\b',
            r'\bAsherman\'s syndrome\b',
            r'\b(?:Uterine|Endometrial) adhesions\b',
            r'\b(?:Septate|Bicornuate) uterus\b',
            
            # Tubal factors
            r'\b(?:Tubal|Fallopian tube) (?:blockage|occlusion|damage)\b',
            r'\b(?:Hydrosalpinx|Salpingitis)\b',
            
            # Male infertility
            r'\b(?:Oligospermia|Azoospermia|Asthenospermia|Teratospermia)\b',
            r'\b(?:Low sperm count|Poor sperm motility|Abnormal sperm morphology)\b',
            r'\bVaricocele\b',
            r'\bObstructive azoospermia\b',
            
            # Miscarriage/recurrent loss
            r'\bRecurrent (?:pregnancy loss|miscarriage|abortion)\b',
            r'\b(?:Habitual|Recurrent) aborter\b',
            
            # Endocrine/hormonal
            r'\bHypothalamic amenorrhea\b',
            r'\b(?:Hyperprolactinemia|Elevated prolactin)\b',
            r'\b(?:Thyroid|Adrenal) disorders affecting fertility\b',
            
            # Your specific examples that are fertility-related
            r'\bfolliculotropic mycosis fungoides\b',
            r'\blocalized pagetoid reticulosis\b',
            r'\bclassic Hodgkin lymphoma, lymphocyte-rich type\b',
            r'\bHodgkin\'s paragranuloma\b',
            r'\bhairy cell leukemia variant\b'
        ],
        "drug": [
            # Ovulation induction
            r'\b(?:Clomiphene citrate|Clomid|Serophene)\b',
            r'\bLetrozole\b',
            r'\b(?:Tamoxifen|Nolvadex)\b',
            
            # Gonadotropins
            r'\b(?:Gonadotropins|Gonadotropin therapy)\b',
            r'\b(?:FSH|follicle-stimulating hormone)\b',
            r'\b(?:LH|luteinizing hormone)\b',
            r'\b(?:hMG|human menopausal gonadotropin)\b',
            r'\b(?:hCG|human chorionic gonadotropin)\b',
            
            # GnRH analogs
            r'\b(?:GnRH agonists|Gonadotropin-releasing hormone agonists)\b',
            r'\b(?:Leuprolide|Lupron)\b',
            r'\b(?:Goserelin|Zoladex)\b',
            r'\b(?:GnRH antagonists|Gonadotropin-releasing hormone antagonists)\b',
            r'\b(?:Ganirelix|Antagon|Cetrorelix|Cetrotide)\b',
            
            # Insulin sensitizers
            r'\bMetformin\b',
            
            # Hormone supplements
            r'\bProgesterone\b',
            r'\b(?:Micronized progesterone|Prometrium|Endometrin|Crinone)\b',
            r'\bEstradiol\b',
            r'\b(?:Estrogen|Estrogen therapy)\b',
            
            # Your specific drug examples (focusing on hormonal/fertility relevance)
            r'\bDiethylstilbestrol\b',
            r'\b(?:Liothyronine|Levothyroxine)\b',  # Thyroid hormones can affect fertility
            r'\b(?:Hydrocortisone|Prednisolone|Betamethasone)\b',  # Corticosteroids for immune issues
            r'\bHydrocortisone (?:cypionate|phosphate|probutate|valerate)\b',
            r'\bPrednisolone (?:phosphate|acetate)\b',
            r'\bBetamethasone phosphate\b',
            
            # Assisted reproduction
            r'\b(?:IVF|in vitro fertilization)\b',
            r'\b(?:ICSI|intracytoplasmic sperm injection)\b',
            r'\b(?:IUI|intrauterine insemination)\b',
            
            # Other fertility treatments
            r'\b(?:Bromocriptine|Parlodel)\b',
            r'\b(?:Cabergoline|Dostinex)\b',
            r'\bDanazol\b',
            r'\b(?:Aspirin|Heparin) for fertility\b'
        ]
    }
    
    # Check if text has fertility context
    text_lower = text.lower()
    fertility_context = any(term in text_lower for term in [
        "fertility", "infertility", "pregnancy", "conception", "reproductive",
        "ovarian", "uterine", "sperm", "egg", "embryo", "ivf", "menstrual"
    ])
    
    # Only extract if fertility context exists
    if not fertility_context:
        return entities
    
    # Extract using patterns
    for entity_type, patterns in fertility_patterns.items():
        for pattern in patterns:
            matches = re.finditer(pattern, text_lower)
            for match in matches:
                # Get the exact text (preserve original case)
                start_pos = match.start()
                end_pos = match.end()
                entity_name = text[start_pos:end_pos]
                
                if not any(e["name"].lower() == entity_name.lower() for e in entities):
                    # Calculate confidence
                    confidence = 0.9  # High confidence for exact pattern matches
                    
                    # Adjust confidence based on specificity
                    if entity_type == "disease":
                        if any(term in entity_name.lower() for term in ["infertility", "pcos", "endometriosis"]):
                            confidence = 0.95
                    elif entity_type == "drug":
                        if any(term in entity_name.lower() for term in ["clomiphene", "letrozole", "gonadotropin"]):
                            confidence = 0.92
                    
                    entities.append({
                        "id": str(len(entities) + 1),
                        "type": entity_type,
                        "name": entity_name,
                        "confidence": confidence,
                        "source": "pattern_match"
                    })
    
    # If no pattern matches but fertility context exists, try keyword-based extraction
    if not entities and fertility_context:
        # Look for capitalized medical terms in fertility context
        words = re.findall(r'\b[A-Z][a-z]+\b', text)
        for i, word in enumerate(words):
            # Check if word looks like a medical term
            if len(word) > 5 and word not in ["Patient", "Treatment", "Therapy", "Doctor"]:
                # Check context around the word
                context_start = max(0, i - 2)
                context_end = min(len(words), i + 3)
                context = " ".join(words[context_start:context_end]).lower()
                
                if any(fert_term in context for fert_term in ["infertility", "fertility", "treatment", "therapy", "medication"]):
                    # Try to determine if it's a disease or drug
                    entity_type = "drug" if any(drug_term in context for drug_term in ["prescribed", "medication", "drug", "therapy"]) else "disease"
                    
                    entities.append({
                        "id": str(len(entities) + 1),
                        "type": entity_type,
                        "name": word,
                        "confidence": 0.6,
                        "source": "context_inference"
                    })
    
    # Deduplicate
    unique_entities = []
    seen_names = set()
    for entity in entities:
        name_lower = entity["name"].lower()
        if name_lower not in seen_names:
            seen_names.add(name_lower)
            unique_entities.append(entity)
    
    return unique_entities

# CORRECTED PROPERTY-BASED SIMILARITY


In [11]:
def calculate_property_similarity(entity_name: str, candidate: Dict, entity_type: str) -> Tuple[float, str, Dict]:
    """
    Calculate enhanced similarity score based on properties.
    Returns: (enhanced_score, match_type, property_info)
    """
    entity_lower = entity_name.lower().strip()
    candidate_props = candidate.get("properties", {})
    
    # Get entity schema
    schema = ENTITY_PROPERTY_SCHEMAS.get(entity_type, {})
    primary_keys = schema.get("primary_keys", [])
    display_props = schema.get("display_properties", [])
    
    # Check for EXACT property matches first (highest priority)
    
    # 1. Check exact primary key matches
    for prop in primary_keys:
        if prop in candidate_props:
            prop_value = str(candidate_props[prop]).lower()
            if prop_value == entity_lower:
                # EXACT PRIMARY KEY MATCH - Highest confidence
                match_type = f"exact_primary_key:{prop}"
                enhanced_score = 0.95  # Very high score for exact primary key
                property_info = {
                    "match_type": match_type,
                    "matched_property": prop,
                    "property_value": candidate_props[prop],
                    "score_breakdown": {"exact_primary_key": PROPERTY_WEIGHTS["exact_primary_key"]}
                }
                return enhanced_score, match_type, property_info
    
    # 2. Check exact display property matches
    for prop in display_props:
        if prop in candidate_props:
            prop_value = str(candidate_props[prop]).lower()
            if prop_value == entity_lower:
                # EXACT DISPLAY PROPERTY MATCH - High confidence
                match_type = f"exact_display_property:{prop}"
                enhanced_score = 0.85
                property_info = {
                    "match_type": match_type,
                    "matched_property": prop,
                    "property_value": candidate_props[prop],
                    "score_breakdown": {"exact_display_property": PROPERTY_WEIGHTS["exact_display_property"]}
                }
                return enhanced_score, match_type, property_info
    
    # 3. Check contains matches in primary keys
    for prop in primary_keys:
        if prop in candidate_props:
            prop_value = str(candidate_props[prop]).lower()
            if entity_lower in prop_value or prop_value in entity_lower:
                # CONTAINS PRIMARY KEY MATCH - Good confidence
                match_type = f"contains_primary_key:{prop}"
                enhanced_score = 0.75
                property_info = {
                    "match_type": match_type,
                    "matched_property": prop,
                    "property_value": candidate_props[prop],
                    "score_breakdown": {"contains_primary_key": PROPERTY_WEIGHTS["contains_primary_key"]}
                }
                return enhanced_score, match_type, property_info
    
    # 4. Check exact name match
    if candidate["node_name"].lower().strip() == entity_lower:
        match_type = "exact_name_match"
        enhanced_score = 0.90
        property_info = {
            "match_type": match_type,
            "matched_property": "node_name",
            "property_value": candidate["node_name"],
            "score_breakdown": {"name_exact": PROPERTY_WEIGHTS["name_exact"]}
        }
        return enhanced_score, match_type, property_info
    
    # 5. Check contains name match
    if entity_lower in candidate["node_name"].lower() or candidate["node_name"].lower() in entity_lower:
        match_type = "contains_name_match"
        enhanced_score = 0.70
        property_info = {
            "match_type": match_type,
            "matched_property": "node_name",
            "property_value": candidate["node_name"],
            "score_breakdown": {"name_contains": PROPERTY_WEIGHTS["name_contains"]}
        }
        return enhanced_score, match_type, property_info
    
    # 6. If no strong property matches, use weighted combination
    base_similarity = candidate["similarity_score"]
    
    # Calculate property boost based on partial matches
    property_score = 0.0
    max_possible_score = 0.0
    matched_properties = []
    
    # Check primary key contains matches (already handled above, but keep for scoring)
    for prop in primary_keys:
        if prop in candidate_props:
            max_possible_score += PROPERTY_WEIGHTS["contains_primary_key"]
            prop_value = str(candidate_props[prop]).lower()
            if entity_lower in prop_value or prop_value in entity_lower:
                property_score += PROPERTY_WEIGHTS["contains_primary_key"]
                matched_properties.append(prop)
    
    # Check display property contains matches
    for prop in display_props:
        if prop in candidate_props:
            max_possible_score += PROPERTY_WEIGHTS["contains_display_property"]
            prop_value = str(candidate_props[prop]).lower()
            if entity_lower in prop_value or prop_value in entity_lower:
                property_score += PROPERTY_WEIGHTS["contains_display_property"]
                matched_properties.append(prop)
    
    # Check other properties
    all_props = schema.get("properties", [])
    other_props = [p for p in all_props if p not in primary_keys and p not in display_props]
    
    for prop in other_props:
        if prop in candidate_props:
            max_possible_score += PROPERTY_WEIGHTS["other_property_match"]
            prop_value = str(candidate_props[prop]).lower()
            if entity_lower in prop_value or prop_value in entity_lower:
                property_score += PROPERTY_WEIGHTS["other_property_match"]
                matched_properties.append(prop)
    
    # Calculate property boost
    if max_possible_score > 0:
        property_boost = property_score / max_possible_score
    else:
        property_boost = 0
    
    # Weighted combination (property matches are more important)
    property_weight = 0.6  # Increased weight for property matches
    text_weight = 1 - property_weight
    
    enhanced_score = (base_similarity * text_weight) + (property_boost * property_weight)
    
    # Ensure score is reasonable
    enhanced_score = min(max(enhanced_score, 0.0), 1.0)
    
    match_type = "property_enhanced" if matched_properties else "text_similarity_only"
    
    property_info = {
        "match_type": match_type,
        "matched_properties": matched_properties,
        "property_boost": property_boost,
        "property_score": property_score,
        "max_possible_score": max_possible_score,
        "score_breakdown": {
            "base_similarity": base_similarity,
            "property_boost": property_boost,
            "final_score": enhanced_score
        }
    }
    
    return enhanced_score, match_type, property_info

def get_enhanced_candidates(entity_name: str, entity_type: str, top_k: int = TOP_K) -> List[Dict]:
    """
    Find top-K most similar KG nodes with property-enhanced matching.
    """
    if not entity_name.strip():
        return []
    
    # First pass: Get candidates based on text similarity
    try:
        query_embedding = embedder.encode([entity_name])[0]
    except:
        return []
    
    # Compute similarity scores
    similarities = cosine_similarity([query_embedding], KG_EMBEDDINGS)[0]
    
    # Get initial candidates
    all_candidates = []
    for idx, score in enumerate(similarities):
        kg_node = KG_NODES[idx]
        candidate = {
            "kg_index": int(idx),
            "node_id": kg_node["node_id"],
            "node_name": kg_node["node_name"],
            "labels": kg_node["labels"],
            "entity_type": kg_node["entity_type"],
            "similarity_score": float(score),
            "properties": kg_node["properties"]
        }
        all_candidates.append(candidate)
    
    # Filter by entity type if specified
    if entity_type != "Unknown":
        type_filtered = [c for c in all_candidates if entity_type == c["entity_type"]]
        if type_filtered:
            all_candidates = type_filtered
    
    # Enhance scores with property matching
    for candidate in all_candidates:
        enhanced_score, match_type, property_info = calculate_property_similarity(
            entity_name, candidate, entity_type
        )
        candidate["enhanced_score"] = enhanced_score
        candidate["match_type"] = match_type
        candidate["property_info"] = property_info
    
    # Sort by enhanced score
    all_candidates.sort(key=lambda x: x["enhanced_score"], reverse=True)
    
    return all_candidates[:top_k]

# CORRECTED ENTITY MAPPING PIPELINE


In [12]:
def stage1_property_exact_match(entity_name: str, entity_type: str, candidates: List[Dict]) -> Optional[Dict]:
    """
    Stage 1: Property-based exact matching.
    """
    entity_lower = entity_name.lower().strip()
    
    for candidate in candidates:
        candidate_props = candidate.get("properties", {})
        
        # Check if entity matches any primary key
        schema = ENTITY_PROPERTY_SCHEMAS.get(entity_type, {})
        primary_keys = schema.get("primary_keys", [])
        
        for prop in primary_keys:
            if prop in candidate_props:
                prop_value = str(candidate_props[prop]).lower()
                if prop_value == entity_lower:
                    candidate["property_match_type"] = f"exact_primary_key:{prop}"
                    candidate["match_type"] = f"exact_primary_key:{prop}"
                    return candidate
        
        # Check if entity matches any display property
        display_props = schema.get("display_properties", [])
        for prop in display_props:
            if prop in candidate_props:
                prop_value = str(candidate_props[prop]).lower()
                if prop_value == entity_lower:
                    candidate["property_match_type"] = f"exact_display_property:{prop}"
                    candidate["match_type"] = f"exact_display_property:{prop}"
                    return candidate
        
        # Check name property
        if candidate["node_name"].lower().strip() == entity_lower:
            candidate["property_match_type"] = "exact_name_match"
            candidate["match_type"] = "exact_name_match"
            return candidate
    
    return None

def stage2_property_enhanced_similarity(candidates: List[Dict]) -> Optional[Dict]:
    """
    Stage 2: Property-enhanced similarity matching.
    """
    if not candidates:
        return None
    
    # Use enhanced score for ranking
    best_candidate = max(candidates, key=lambda x: x.get("enhanced_score", 0))
    
    threshold = SIM_THRESHOLD
    
    # Adjust threshold based on match type
    match_type = best_candidate.get("match_type", "")
    if any(x in match_type for x in ["exact_primary_key", "exact_display_property", "exact_name_match"]):
        threshold = 0.8  # Lower threshold for exact matches
    elif "contains" in match_type:
        threshold = 0.6
    elif best_candidate.get("enhanced_score", 0) > 0.7:
        threshold = 0.65  # Adjust for high scores
    
    if best_candidate.get("enhanced_score", 0) >= threshold:
        return best_candidate
    
    return None

def stage3_property_contextual_llm_match(entity: Dict, candidates: List[Dict], question: str, client) -> Optional[Dict]:
    """
    Stage 3: LLM-based matching with biomedical property context.
    """
    if not candidates:
        return None
    
    # Prepare detailed candidate information with properties
    candidates_info = []
    for i, candidate in enumerate(candidates[:5]):
        candidate_info = {
            "index": i,
            "node_name": candidate["node_name"],
            "labels": candidate["labels"],
            "entity_type": candidate.get("entity_type", "Unknown"),
            "similarity_score": round(candidate["similarity_score"], 4),
            "enhanced_score": round(candidate.get("enhanced_score", 0), 4),
            "match_type": candidate.get("match_type", "unknown")
        }
        
        # Add relevant biomedical properties
        candidate_props = candidate.get("properties", {})
        schema = ENTITY_PROPERTY_SCHEMAS.get(candidate_info["entity_type"], {})
        display_props = schema.get("display_properties", [])
        
        # Include important biomedical properties
        important_props = {}
        
        # For disease entities
        if candidate_info["entity_type"] == "disease":
            disease_props = ["mondo_definitions", "mayo_symptoms", "orphanet_prevalence", 
                           "SNOMEDCT_US_definition", "orphanet_definition", "mayo_causes"]
            for prop in disease_props:
                if prop in candidate_props and candidate_props[prop]:
                    # Truncate long text for display
                    prop_value = str(candidate_props[prop])
                    if len(prop_value) > 150:
                        prop_value = prop_value[:150] + "..."
                    important_props[prop] = prop_value
                    if len(important_props) >= 3:
                        break
        
        # For drug entities  
        elif candidate_info["entity_type"] == "drug":
            drug_props = ["description", "indication", "mechanism_of_action", 
                         "category", "atc_4", "pharmacodynamics"]
            for prop in drug_props:
                if prop in candidate_props and candidate_props[prop]:
                    prop_value = str(candidate_props[prop])
                    if len(prop_value) > 150:
                        prop_value = prop_value[:150] + "..."
                    important_props[prop] = prop_value
                    if len(important_props) >= 3:
                        break
        
        # Include primary keys
        for prop in schema.get("primary_keys", [])[:2]:
            if prop in candidate_props and prop not in important_props:
                important_props[prop] = candidate_props[prop]
        
        candidate_info["key_properties"] = important_props
        candidates_info.append(candidate_info)
    
    # Create biomedical property-based reasoning prompt
    llm_prompt = f"""
    You are a biomedical knowledge graph expert specializing in fertility medicine. 
    Help map a query entity to the most relevant knowledge graph node.
    
    CONTEXT:
    - Question: "{question}"
    - Query Entity: "{entity['name']}" (Type: {entity.get('type', 'Unknown')})
    - Entity Confidence: {entity.get('confidence', 0.7):.2f}
    - Domain: Fertility/Reproductive Medicine
    
    CANDIDATE NODES (with key biomedical properties):
    {json.dumps(candidates_info, indent=2)}
    
    IMPORTANT PROPERTY MAPPINGS FOR FERTILITY ENTITIES:
    
    1. DISEASE NODES:
       - Primary keys: node_id, node_name
       - Key properties to check: 
         * mondo_definitions: Medical definitions from MONDO ontology
         * mayo_symptoms: Clinical symptoms from Mayo Clinic
         * orphanet_prevalence: Prevalence in population
         * SNOMEDCT_US_definition: Standard medical terminology definitions
         * orphanet_definition: Rare disease definitions
    
    2. DRUG NODES:
       - Primary keys: node_id, node_name  
       - Key properties to check:
         * description: General description of the drug
         * indication: What conditions it treats
         * mechanism_of_action: How it works biologically
         * category: Drug classification/category
         * atc_4: Anatomical Therapeutic Chemical classification
    
    ANALYSIS INSTRUCTIONS FOR FERTILITY CONTEXT:
    1. Check if query entity matches any PRIMARY KEY values exactly
    2. Check if query entity appears in DISPLAY PROPERTY values
    3. Evaluate FERTILITY RELEVANCE - does the entity relate to reproduction?
    4. Consider if properties mention fertility-related terms:
       - For diseases: infertility, pregnancy, menstrual, ovarian, sperm, embryo
       - For drugs: fertility treatment, ovulation induction, hormone therapy
    5. Evaluate the enhanced score (combines similarity + property matching)
    6. If NO candidate is suitable, select NONE
    
    OUTPUT FORMAT:
    {{
      "selected_entity": {{
        "index": selected_index_or_-1_for_none,
        "node_name": "selected_node_name_or_NONE",
        "matching_properties": ["property1", "property2"],
        "reason": "brief_explanation_focusing_on_property_matches_and_fertility_relevance"
      }}
    }}
    """
    
    try:
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "You select the most relevant biomedical knowledge graph entity based on property matching and fertility context. Focus on reproductive medicine relevance."},
                {"role": "user", "content": llm_prompt}
            ],
            temperature=0.2,
            max_tokens=500
        )
        
        content = response.choices[0].message.content
        parsed = safe_json_parse(content)
        
        # Safely access the selected_entity
        selected = {}
        if isinstance(parsed, dict):
            selected = parsed.get("selected_entity", {})
        elif isinstance(parsed, list) and len(parsed) > 0:
            selected = parsed[0] if isinstance(parsed[0], dict) else {}
        
        # Safely get index
        selected_index = -1
        if isinstance(selected, dict):
            selected_index = selected.get("index", -1)
        else:
            # Try to extract index from string
            if isinstance(selected, str) and "index" in selected.lower():
                match = re.search(r'index["\']?\s*:\s*(-?\d+)', selected, re.IGNORECASE)
                if match:
                    selected_index = int(match.group(1))
        
        if 0 <= selected_index < len(candidates):
            candidate = candidates[selected_index]
            
            # Safely get matching_properties
            matching_properties = []
            if isinstance(selected, dict):
                matching_properties = selected.get("matching_properties", [])
                if not isinstance(matching_properties, list):
                    matching_properties = []
            
            # Safely get reason
            reason = "LLM selected based on biomedical property context"
            if isinstance(selected, dict):
                reason = selected.get("reason", reason)
            
            candidate["llm_property_matches"] = matching_properties
            candidate["llm_reason"] = reason
            return candidate
    
    except Exception as e:
        print(f"❌ Error in LLM property matching: {e}")
        if 'content' in locals():
            print(f"   Content: {content[:200]}...")
    
    return None

def map_entity_to_kg_with_properties(entity: Dict, question: str, client) -> Dict:
    """
    Enhanced entity mapping using property-aware pipeline.
    """
    entity_name = entity["name"]
    entity_type = entity.get("type", "Unknown")
    
    # Stage 0: Get property-enhanced candidates
    candidates = get_enhanced_candidates(entity_name, entity_type, TOP_K)
    
    if not candidates:
        return {
            "query_entity": entity,
            "mapped_node": None,
            "mapping_stage": "no_candidates",
            "reason": "No similar nodes found in knowledge graph",
            "property_info": {}
        }
    
    # Stage 1: Property-based exact match
    exact_match = stage1_property_exact_match(entity_name, entity_type, candidates)
    if exact_match:
        return {
            "query_entity": entity,
            "mapped_node": {
                "node_id": exact_match["node_id"],
                "node_name": exact_match["node_name"],
                "entity_type": exact_match.get("entity_type", "Unknown"),
                "similarity_score": exact_match["similarity_score"],
                "enhanced_score": exact_match.get("enhanced_score", 0),
                "match_type": exact_match.get("match_type", "unknown"),
                "property_match_type": exact_match.get("property_match_type", "unknown"),
                "mapping_stage": "property_exact",
                "reason": f"Property-based exact match: {exact_match.get('property_match_type', 'unknown')}"
            },
            "property_info": exact_match.get("property_info", {})
        }
    
    # Stage 2: Property-enhanced similarity
    similarity_match = stage2_property_enhanced_similarity(candidates)
    if similarity_match:
        return {
            "query_entity": entity,
            "mapped_node": {
                "node_id": similarity_match["node_id"],
                "node_name": similarity_match["node_name"],
                "entity_type": similarity_match.get("entity_type", "Unknown"),
                "similarity_score": similarity_match["similarity_score"],
                "enhanced_score": similarity_match.get("enhanced_score", 0),
                "match_type": similarity_match.get("match_type", "unknown"),
                "mapping_stage": "property_enhanced",
                "reason": f"Property-enhanced similarity ({similarity_match.get('match_type', 'unknown')}, score: {similarity_match.get('enhanced_score', 0):.4f})"
            },
            "property_info": similarity_match.get("property_info", {})
        }
    
    # Stage 3: LLM with property context
    llm_match = stage3_property_contextual_llm_match(entity, candidates, question, client)
    if llm_match:
        return {
            "query_entity": entity,
            "mapped_node": {
                "node_id": llm_match["node_id"],
                "node_name": llm_match["node_name"],
                "entity_type": llm_match.get("entity_type", "Unknown"),
                "similarity_score": llm_match["similarity_score"],
                "enhanced_score": llm_match.get("enhanced_score", 0),
                "match_type": llm_match.get("match_type", "unknown"),
                "mapping_stage": "property_llm",
                "reason": llm_match.get("llm_reason", "LLM selected based on property context"),
                "llm_property_matches": llm_match.get("llm_property_matches", [])
            },
            "property_info": llm_match.get("property_info", {})
        }
    
    # Fallback: Return best candidate
    best_candidate = candidates[0]
    return {
        "query_entity": entity,
        "mapped_node": {
            "node_id": best_candidate["node_id"],
            "node_name": best_candidate["node_name"],
            "entity_type": best_candidate.get("entity_type", "Unknown"),
            "similarity_score": best_candidate["similarity_score"],
            "enhanced_score": best_candidate.get("enhanced_score", 0),
            "match_type": best_candidate.get("match_type", "unknown"),
            "mapping_stage": "property_fallback",
            "reason": f"No strong property match found. Returning best enhanced match: {best_candidate['node_name']}"
        },
        "property_info": best_candidate.get("property_info", {})
    }


# PATH SEARCH AND PRUNING FUNCTIONS


In [13]:
def find_paths_between_nodes(source_node_id: str, target_node_id: str, max_length: int = MAX_PATH_LENGTH) -> List[Dict]:
    """
    Find all paths between two nodes in the knowledge graph.
    Returns paths as sequences of nodes and relationships.
    """
    with driver.session() as session:
        try:
            # Find all paths up to max_length hops
            result = session.run("""
            MATCH path = (a)-[*1..%d]-(b)
            WHERE elementId(a) = $source_id AND elementId(b) = $target_id
            RETURN path, length(path) as pathLength
            ORDER BY pathLength ASC
            LIMIT 20
            """ % max_length,
            source_id=source_node_id, target_id=target_node_id)
            
            paths = []
            seen_paths = set()
            
            for record in result:
                path = record["path"]
                path_length = record["pathLength"]
                
                # Extract nodes and relationships from path
                nodes_in_path = []
                relationships_in_path = []
                
                # Get nodes
                for node in path.nodes:
                    node_props = dict(node)
                    node_labels = list(node.labels)
                    node_id = str(node.element_id)
                    
                    # Get node name for display - biomedical context
                    node_name = None
                    
                    # Try biomedical property names first
                    for prop_name in [
    # Disease-specific properties
    "SNOMEDCT_US_definition",
    "mayo_causes",
    "mayo_complications",
    "mayo_prevention",
    "mayo_risk_factors",
    "mayo_see_doc",
    "mayo_symptoms",
    "mondo_definitions",
    "orphanet_clinical_description",
    "orphanet_definition",
    "orphanet_epidemiology",
    "orphanet_management_and_treatment",
    "orphanet_prevalence",
    "umls_descriptions",
    
    # Drug-specific properties
    "atc_4",
    "category",
    "clogp",
    "description",
    "group",
    "half_life",
    "indication",
    "mechanism_of_action",
    "molecular_weight",
    "pathway",
    "pharmacodynamics",
    "protein_binding",
    "state",
    "tpsa",
    
    # Common properties (appear in both)
    "node_id",
    "node_index",
    "node_name",
    "node_source"
]:
                        if prop_name in node_props and node_props[prop_name]:
                            node_name = str(node_props[prop_name])
                            # Truncate if too long
                            if len(node_name) > 100:
                                node_name = node_name[:100] + "..."
                            break
                    
                    # Fallback to any property
                    if not node_name and node_props:
                        for prop_value in node_props.values():
                            if prop_value:
                                node_name = str(prop_value)
                                if len(node_name) > 100:
                                    node_name = node_name[:100] + "..."
                                break
                    
                    nodes_in_path.append({
                        "id": node_id,
                        "labels": node_labels,
                        "name": node_name or f"Node_{node_id}",
                        "properties": node_props
                    })
                
                # Get relationships
                for rel in path.relationships:
                    rel_type = rel.type
                    rel_props = dict(rel)
                    
                    relationships_in_path.append({
                        "type": rel_type,
                        "properties": rel_props
                    })
                
                # Create path string representation
                path_string_parts = []
                for i, node in enumerate(nodes_in_path):
                    path_string_parts.append(node["name"])
                    if i < len(relationships_in_path):
                        rel = relationships_in_path[i]
                        path_string_parts.append(f"--[{rel['type']}]-->")
                
                path_string = " ".join(path_string_parts)
                
                # Create unique signature to avoid duplicates
                path_signature = path_string
                
                if path_signature not in seen_paths:
                    seen_paths.add(path_signature)
                    
                    paths.append({
                        "path_string": path_string,
                        "path_length": path_length,
                        "nodes": nodes_in_path,
                        "relationships": relationships_in_path,
                        "source_node": nodes_in_path[0],
                        "target_node": nodes_in_path[-1],
                        "detailed_description": generate_detailed_path_description(nodes_in_path, relationships_in_path)
                    })
            
            return paths
            
        except Exception as e:
            print(f"❌ Error finding paths: {e}")
            return []

def generate_detailed_path_description(nodes: List[Dict], relationships: List[Dict]) -> str:
    """Generate a natural language description of the path."""
    description_parts = []
    
    for i, (node, rel) in enumerate(zip(nodes[:-1], relationships)):
        # Node description - biomedical context
        labels_str = ', '.join(node['labels'])
        if not labels_str:
            labels_str = "Unknown"
        node_desc = f"{node['name']} ({labels_str})"
        
        # Relationship description - keep biomedical terms
        rel_desc = rel['type'].lower().replace('_', ' ')
        
        description_parts.append(f"{node_desc} {rel_desc}")
    
    # Add last node
    if nodes:
        last_node = nodes[-1]
        labels_str = ', '.join(last_node['labels'])
        if not labels_str:
            labels_str = "Unknown"
        last_node_desc = f"{last_node['name']} ({labels_str})"
        description_parts.append(last_node_desc)
    
    return " → ".join(description_parts)

def prune_paths_with_llm(question: str, all_paths: List[Dict], entity_mappings: List[Dict], client) -> Dict:
    """
    Use LLM to select the most relevant path for answering the question.
    """
    if not all_paths:
        return {"selected_path": None, "reasoning": "No paths found"}
    
    # Prepare entity mapping information
    entity_info = []
    for mapping in entity_mappings:
        if mapping.get("mapped_node"):
            entity = mapping["query_entity"]
            mapped = mapping["mapped_node"]
            entity_info.append({
                "query_entity": entity["name"],
                "entity_type": entity.get("type", "Unknown"),
                "mapped_to": mapped["node_name"],
                "mapping_confidence": mapped.get("enhanced_score", 0),
                "mapping_stage": mapped.get("mapping_stage", "unknown")
            })
    
    # Prepare path candidates for LLM evaluation
    path_candidates = []
    for i, path in enumerate(all_paths[:TOP_PATHS_FOR_PRUNING]):
        # Calculate path relevance score (combine length and entity coverage)
        entity_coverage = 0
        for mapping in entity_mappings:
            if mapping.get("mapped_node"):
                mapped_name = mapping["mapped_node"]["node_name"]
                # Check if this entity appears in the path
                for node in path["nodes"]:
                    if node["name"] == mapped_name:
                        entity_coverage += 1
                        break
        
        path_relevance_score = entity_coverage / max(len(entity_mappings), 1)
        
        # Calculate biomedical relevance score based on labels
        biomedical_relevance = 0
        fertility_keywords = ["disease", "drug", "gene", "protein", "pathway", 
                            "treatment", "therapy", "infertility", "fertility"]
        
        for node in path["nodes"]:
            labels_lower = [label.lower() for label in node["labels"]]
            for keyword in fertility_keywords:
                if any(keyword in label for label in labels_lower):
                    biomedical_relevance += 1
                    break
        
        path_candidates.append({
            "path_id": i + 1,
            "path_string": path["path_string"],
            "path_length": path["path_length"],
            "detailed_description": path.get("detailed_description", path["path_string"]),
            "entity_coverage": entity_coverage,
            "relevance_score": round(path_relevance_score, 2),
            "biomedical_relevance": biomedical_relevance
        })
    
    # Create LLM prompt for path pruning - BIOMEDICAL CONTEXT
    pruning_prompt = f"""
    You are a biomedical knowledge graph expert specializing in fertility and reproductive medicine. 
    Select the most relevant path for answering the given clinical question.

    CLINICAL QUESTION: "{question}"

    ENTITY MAPPINGS (from question to knowledge graph):
    {json.dumps(entity_info, indent=2)}

    AVAILABLE PATHS in knowledge graph:
    {json.dumps(path_candidates, indent=2)}

    SELECTION CRITERIA FOR BIOMEDICAL CONTEXT:
    1. CLINICAL RELEVANCE: Does the path directly address the clinical question about fertility?
    2. ENTITY COVERAGE: Does the path include all/most mapped clinical entities?
    3. BIOMEDICAL COHERENCE: Does the path make biological/medical sense?
    4. PATH LENGTH: Shorter paths are preferred for clarity
    5. FERTILITY CONTEXT: Does the path involve fertility-related concepts?
        - Disease-drug relationships
        - Treatment pathways
        - Biological mechanisms
        - Clinical outcomes

    OUTPUT FORMAT:
    {{
      "selected_path": {{
        "path_id": selected_path_number,
        "path_string": "selected_path_string",
        "reasoning": "detailed_explanation_focusing_on_biomedical_relevance_and_fertility_context",
        "confidence_score": 0.0_to_1.0
      }}
    }}

    If no path is clinically relevant, set "path_id" to -1.
    """
    
    try:
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "You select the most relevant biomedical knowledge graph path for answering fertility-related clinical questions. Focus on clinical relevance and biological plausibility."},
                {"role": "user", "content": pruning_prompt}
            ],
            temperature=0.1,
            max_tokens=600
        )
        
        content = response.choices[0].message.content
        parsed = safe_json_parse(content)
        
        if not parsed:
            # Fallback: select shortest path with best entity coverage
            return select_best_path_fallback(path_candidates, entity_mappings)
        
        selected_path = parsed.get("selected_path", {})
        path_id = selected_path.get("path_id", -1)
        
        if 1 <= path_id <= len(all_paths):
            selected_path_data = all_paths[path_id - 1]
            
            # Add LLM reasoning to the path
            selected_path_data["llm_selected"] = True
            selected_path_data["selection_reasoning"] = selected_path.get("reasoning", "LLM selected")
            selected_path_data["selection_confidence"] = selected_path.get("confidence_score", 0.8)
            
            return {
                "selected_path": selected_path_data,
                "reasoning": selected_path.get("reasoning", "Path selected by LLM for clinical relevance"),
                "confidence": selected_path.get("confidence_score", 0.8),
                "all_considered_paths": path_candidates
            }
        else:
            return select_best_path_fallback(path_candidates, entity_mappings)
            
    except Exception as e:
        print(f"❌ Error in LLM path pruning: {e}")
        return select_best_path_fallback(path_candidates, entity_mappings)

def select_best_path_fallback(path_candidates: List[Dict], entity_mappings: List[Dict]) -> Dict:
    """Fallback method to select best path when LLM fails."""
    if not path_candidates:
        return {"selected_path": None, "reasoning": "No paths available"}
    
    # Score each path
    scored_paths = []
    for path in path_candidates:
        score = 0
        
        # Preference for shorter paths
        length_score = 1.0 / (path["path_length"] + 1)
        
        # Preference for higher entity coverage
        coverage_score = path["entity_coverage"] / max(len(entity_mappings), 1)
        
        # Preference for biomedical relevance
        biomedical_score = path.get("biomedical_relevance", 0) / max(len(path.get("nodes", [])), 1)
        
        # Combined score - weighted
        total_score = (length_score * 0.3) + (coverage_score * 0.4) + (biomedical_score * 0.3)
        
        scored_paths.append((total_score, path))
    
    # Select best path
    scored_paths.sort(key=lambda x: x[0], reverse=True)
    best_score, best_path = scored_paths[0]
    
    return {
        "selected_path": best_path,
        "reasoning": f"Fallback: Selected path with best entity coverage ({best_path['entity_coverage']}/{len(entity_mappings)}), biomedical relevance ({best_path.get('biomedical_relevance', 0)}), and length {best_path['path_length']}",
        "confidence": round(best_score, 2),
        "all_considered_paths": path_candidates
    }

def find_shortest_paths_between_entities(mapped_entities: List[Dict]) -> Dict:
    """
    Find shortest paths between all pairs of successfully mapped entities.
    """
    path_results = {
        "entity_pairs": [],
        "total_paths_found": 0,
        "all_paths": []
    }
    
    # Get successfully mapped nodes
    successful_mappings = [
        mapping for mapping in mapped_entities 
        if mapping.get("mapped_node") and mapping["mapped_node"].get("node_id")
    ]
    
    if len(successful_mappings) < 2:
        return path_results
    
    print(f"\n🔍 Searching paths between {len(successful_mappings)} mapped biomedical entities...")
    
    # Find paths for each pair
    for i in range(len(successful_mappings)):
        for j in range(i + 1, len(successful_mappings)):
            mapping1 = successful_mappings[i]
            mapping2 = successful_mappings[j]
            
            source_id = mapping1["mapped_node"]["node_id"]
            target_id = mapping2["mapped_node"]["node_id"]
            source_name = mapping1["query_entity"]["name"]
            target_name = mapping2["query_entity"]["name"]
            source_type = mapping1["query_entity"].get("type", "Unknown")
            target_type = mapping2["query_entity"].get("type", "Unknown")
            
            print(f"  🔗 {source_name} ({source_type}) → {target_name} ({target_type})...")
            
            paths = find_paths_between_nodes(source_id, target_id)
            
            if paths:
                # Sort by path length and take top N
                paths.sort(key=lambda x: x["path_length"])
                top_paths = paths[:MAX_PATHS_PER_PAIR]
                
                pair_result = {
                    "source_entity": source_name,
                    "target_entity": target_name,
                    "source_type": source_type,
                    "target_type": target_type,
                    "source_node": mapping1["mapped_node"]["node_name"],
                    "target_node": mapping2["mapped_node"]["node_name"],
                    "paths_found": len(paths),
                    "shortest_path_length": paths[0]["path_length"] if paths else None,
                    "top_paths": top_paths
                }
                
                path_results["entity_pairs"].append(pair_result)
                path_results["total_paths_found"] += len(paths)
                path_results["all_paths"].extend(top_paths)
                
                print(f"    ✅ Found {len(paths)} paths (showing {len(top_paths)})")
                for path in top_paths[:2]:  # Show first 2 paths
                    print(f"       • {path['path_string']}")
            else:
                print(f"    ❌ No paths found")
    
    return path_results

# CHAIN-OF-THOUGHT (COT) GENERATION 

In [14]:
def generate_chain_of_thought(question: str, selected_path: Dict, entity_mappings: List[Dict], client) -> Dict:
    """
    Generate Chain-of-Thought reasoning based on the selected biomedical path.
    """
    if not selected_path:
        return {
            "cot": "No relevant path found to generate reasoning.",
            "reasoning_steps": [],
            "final_answer": "Cannot answer based on available biomedical knowledge graph information."
        }
    
    # Extract path information
    path_description = selected_path.get("detailed_description", selected_path["path_string"])
    nodes = selected_path.get("nodes", [])
    relationships = selected_path.get("relationships", [])
    
    # Prepare entity context
    entity_context = []
    for mapping in entity_mappings:
        if mapping.get("mapped_node"):
            entity = mapping["query_entity"]
            mapped = mapping["mapped_node"]
            entity_context.append({
                "mentioned_in_question": entity["name"],
                "mapped_to_knowledge_graph": mapped["node_name"],
                "entity_type": entity.get("type", "Unknown"),
                "mapping_confidence": round(mapped.get("enhanced_score", 0), 3)
            })
    
    # Prepare path nodes information for biomedical context
    path_nodes_info = []
    for i, node in enumerate(nodes):
        node_info = {
            "step": i + 1,
            "node_name": node["name"],
            "node_labels": node["labels"],
            "role_in_path": "Source" if i == 0 else "Target" if i == len(nodes)-1 else "Intermediate"
        }
        
        # Add biomedical context based on labels and properties
        props = node.get("properties", {})
        
        # Check for disease properties
        disease_props = ["mondo_definitions", "mayo_symptoms", "orphanet_definition"]
        for prop in disease_props:
            if prop in props:
                node_info["disease_info"] = f"Has {prop}: {str(props[prop])[:100]}..."
                break
        
        # Check for drug properties
        drug_props = ["description", "indication", "mechanism_of_action"]
        for prop in drug_props:
            if prop in props:
                node_info["drug_info"] = f"Has {prop}: {str(props[prop])[:100]}..."
                break
        
        # Add fertility relevance
        fertility_keywords = ["fertility", "infertility", "ovarian", "uterine", "sperm", "egg", "embryo", "pregnancy"]
        node_text = json.dumps(node).lower()
        fertility_mentions = [kw for kw in fertility_keywords if kw in node_text]
        if fertility_mentions:
            node_info["fertility_relevance"] = f"Mentions: {', '.join(fertility_mentions[:3])}"
        
        path_nodes_info.append(node_info)
    
    # Prepare relationships information
    relationships_info = []
    for i, rel in enumerate(relationships):
        rel_info = {
            "step": i + 1,
            "relationship_type": rel["type"],
            "biomedical_meaning": translate_biomedical_relationship(rel["type"]),
            "connects": f"{nodes[i]['name']} → {nodes[i+1]['name']}"
        }
        
        # Check if relationship has fertility relevance
        rel_text = json.dumps(rel).lower()
        fertility_rel_keywords = ["treats", "causes", "associated", "targets", "interacts", "regulates"]
        if any(kw in rel_text for kw in fertility_rel_keywords):
            rel_info["fertility_context"] = "May relate to fertility mechanisms"
            
        relationships_info.append(rel_info)
    
    # Create CoT generation prompt for biomedical context
    cot_prompt = f"""
    CLINICAL QUESTION: "{question}"

    ENTITY MAPPING CONTEXT (question entities mapped to knowledge graph):
    {json.dumps(entity_context, indent=2)}

    SELECTED BIOMEDICAL KNOWLEDGE GRAPH PATH:
    Path Description: {path_description}
    
    Detailed Path Analysis:
    1. PATH NODES (biomedical entities):
    {json.dumps(path_nodes_info, indent=2)}
    
    2. BIOLOGICAL RELATIONSHIPS:
    {json.dumps(relationships_info, indent=2)}

    TASK: Generate a Chain-of-Thought (CoT) reasoning to answer the clinical question based on the biomedical knowledge graph path.

    COT STRUCTURE FOR BIOMEDICAL CONTEXT:
    1. CLINICAL ENTITY IDENTIFICATION: Identify which clinical entities from the question are present in the path
    2. PATH BIOLOGICAL INTERPRETATION: Explain what the path means in biological/medical terms
    3. RELATIONSHIP ANALYSIS: Analyze each biological relationship in the context of fertility/medicine
    4. CLINICAL INFERENCE: Draw logical clinical conclusions from the path
    5. ANSWER: Provide a concise clinical answer based on the reasoning

    IMPORTANT FOR FERTILITY CONTEXT:
    - Focus on reproductive health implications
    - Consider disease-drug relationships
    - Mention fertility relevance when applicable
    - Base reasoning ONLY on the specific path shown above

    OUTPUT FORMAT:
    {{
      "chain_of_thought": {{
        "reasoning_steps": [
          "Step 1: Identify clinical entities from question that appear in the path",
          "Step 2: Explain the first biological relationship in medical terms",
          "Step 3: Explain subsequent biological relationships",
          "Step 4: Draw clinical conclusions based on the complete path",
          "Step 5: Formulate the clinical answer"
        ],
        "detailed_reasoning": "Multi-paragraph detailed explanation connecting the path to the clinical question",
        "final_answer": "Concise clinical answer to the original question"
      }}
    }}
    """
    
    try:
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "You are a clinical researcher that generates detailed Chain-of-Thought reasoning based on biomedical knowledge graph paths. Specialize in fertility and reproductive medicine. Be precise and reference specific biological entities and relationships."},
                {"role": "user", "content": cot_prompt}
            ],
            temperature=0.2,
            max_tokens=800
        )
        
        content = response.choices[0].message.content
        parsed = safe_json_parse(content)
        
        if not parsed:
            # Generate fallback CoT
            return generate_fallback_biomedical_cot(question, selected_path, entity_mappings)
        
        cot_data = parsed.get("chain_of_thought", {})
        
        # Ensure we have the required structure
        if not cot_data.get("reasoning_steps"):
            cot_data["reasoning_steps"] = [
                f"Step 1: The path connects {selected_path['source_node']['name']} to {selected_path['target_node']['name']}",
                f"Step 2: The biological relationship is {selected_path['relationships'][0]['type'] if selected_path.get('relationships') else 'unknown'}",
                "Step 3: This indicates a biomedical connection between the entities",
                "Step 4: Based on this connection, we can infer clinical implications",
                "Step 5: Clinical answer formulated from path analysis"
            ]
        
        if not cot_data.get("detailed_reasoning"):
            cot_data["detailed_reasoning"] = generate_biomedical_path_based_reasoning(selected_path, question)
        
        if not cot_data.get("final_answer"):
            cot_data["final_answer"] = generate_biomedical_final_answer(selected_path, question)
        
        return cot_data
        
    except Exception as e:
        print(f"❌ Error in CoT generation: {e}")
        return generate_fallback_biomedical_cot(question, selected_path, entity_mappings)

def translate_biomedical_relationship(rel_type: str) -> str:
    """Translate relationship type to biomedical meaning."""
    translations = {
        "ASSOCIATED_WITH": "is clinically associated with",
        "TREATS": "is used to treat",
        "CAUSES": "can cause or contribute to",
        "INTERACTS_WITH": "biologically interacts with",
        "REGULATES": "regulates or modulates",
        "TARGETS": "targets or affects",
        "INDUCES": "induces or triggers",
        "INHIBITS": "inhibits or blocks",
        "ENHANCES": "enhances or increases",
        "PREDISPOSES": "predisposes to or increases risk of",
        "PART_OF": "is part of biological pathway",
        "ENCODES": "encodes or produces",
        "METABOLIZES": "metabolizes or breaks down",
        "BINDS_TO": "binds to or interacts with",
        "UPREGULATES": "upregulates or increases expression of",
        "DOWNREGULATES": "downregulates or decreases expression of"
    }
    return translations.get(rel_type, f"has biological relationship: {rel_type.lower().replace('_', ' ')}")

def generate_biomedical_path_based_reasoning(path: Dict, question: str) -> str:
    """Generate clinical reasoning based on path structure."""
    if not path:
        return "No path available for clinical reasoning."
    
    path_desc = path.get("detailed_description", path.get("path_string", ""))
    nodes = path.get("nodes", [])
    
    reasoning = f"Analyzing the biomedical path: {path_desc}\n\n"
    
    if len(nodes) >= 2:
        # Extract actual node names
        node_names = []
        for node in nodes:
            name = node.get("name", "Unknown")
            # Clean up the name
            name = name.split(" --[")[0] if " --[" in name else name
            node_names.append(name)
        
        reasoning += f"The clinical path connects {node_names[0]} "
        
        for i, rel in enumerate(path.get("relationships", [])):
            rel_meaning = translate_biomedical_relationship(rel['type'])
            reasoning += f"which {rel_meaning} {node_names[i+1]} "
        
        reasoning += f"\n\nThis path shows a direct biomedical relationship between the clinical entities mentioned in the question."
        
        # Add fertility context if relevant
        question_lower = question.lower()
        path_text = json.dumps(path).lower()
        fertility_terms = ["fertility", "infertility", "pregnancy", "reproductive", "ovarian", "sperm"]
        
        if any(term in question_lower for term in fertility_terms) or any(term in path_text for term in fertility_terms):
            reasoning += " This relationship is particularly relevant to fertility and reproductive health."
    
    return reasoning

def generate_biomedical_final_answer(path: Dict, question: str) -> str:
    """Generate clinical final answer based on path."""
    if not path:
        return "Based on the available biomedical knowledge graph, no direct clinical relationship was found to answer the question."
    
    # Get the path string and nodes
    path_string = path.get("path_string", "")
    nodes = path.get("nodes", [])
    
    # Extract source and target node names
    source_name = "Unknown"
    target_name = "Unknown"
    
    if nodes:
        source_name = nodes[0].get("name", "Unknown") if nodes else "Unknown"
        target_name = nodes[-1].get("name", "Unknown") if nodes else "Unknown"
    
    # Clean up node names
    source_name = source_name.split(" --[")[0] if " --[" in source_name else source_name
    target_name = target_name.split(" --[")[0] if " --[" in target_name else target_name
    
    # Get relationship information
    relationships = path.get("relationships", [])
    rel_info = ""
    if relationships:
        rel_type = relationships[0].get("type", "connected")
        rel_meaning = translate_biomedical_relationship(rel_type)
        rel_info = f" {rel_meaning} "
    
    # Generate appropriate answer based on question
    question_lower = question.lower()
    
    # Fertility-specific patterns
    if any(term in question_lower for term in ["fertility", "infertility", "reproductive"]):
        if "drug" in question_lower or "treatment" in question_lower or "treat" in question_lower:
            return f"Yes, {source_name}{rel_info}{target_name} in the context of fertility treatment."
        elif "cause" in question_lower or "risk" in question_lower:
            return f"Yes, {source_name}{rel_info}{target_name}, which may impact fertility."
        else:
            return f"Yes, there is a biomedical relationship between {source_name} and {target_name} relevant to fertility."
    
    # Disease-drug patterns
    elif "disease" in question_lower and "drug" in question_lower:
        if "treat" in question_lower or "effective" in question_lower:
            return f"Yes, {target_name}{rel_info}{source_name} for treatment."
        elif "side effect" in question_lower or "risk" in question_lower:
            return f"Yes, {source_name}{rel_info}{target_name}, which may have clinical implications."
        else:
            return f"Yes, {source_name}{rel_info}{target_name} in the knowledge graph."
    
    # General biomedical relationship
    elif "relationship" in question_lower or "connect" in question_lower or "associate" in question_lower:
        return f"Yes, {source_name}{rel_info}{target_name} in the biomedical knowledge graph."
    
    else:
        # Generic biomedical answer
        return f"Yes, {source_name} is biologically connected to {target_name} in the knowledge graph."

def generate_fallback_biomedical_cot(question: str, selected_path: Dict, entity_mappings: List[Dict]) -> Dict:
    """Generate fallback biomedical Chain-of-Thought when LLM fails."""
    if not selected_path:
        return {
            "chain_of_thought": {
                "reasoning_steps": ["No clinical path available for reasoning"],
                "detailed_reasoning": "Cannot generate clinical reasoning without a valid biomedical knowledge graph path.",
                "final_answer": "Insufficient information in biomedical knowledge graph to answer the clinical question."
            }
        }
    
    # Get node names from the path
    nodes = selected_path.get("nodes", [])
    source_name = "Unknown"
    target_name = "Unknown"
    
    if nodes:
        source_name = nodes[0].get("name", "Unknown")
        target_name = nodes[-1].get("name", "Unknown")
        # Clean names
        source_name = source_name.split(" --[")[0] if " --[" in source_name else source_name
        target_name = target_name.split(" --[")[0] if " --[" in target_name else target_name
    
    # Get entity names from mappings
    entity_names = []
    for mapping in entity_mappings:
        if mapping.get("query_entity"):
            entity_names.append(mapping["query_entity"]["name"])
    
    path_desc = selected_path.get("detailed_description", selected_path.get("path_string", ""))
    
    # Get relationship information for biomedical context
    relationships = selected_path.get("relationships", [])
    rel_text = ""
    if relationships:
        rel_type = relationships[0].get("type", "connected")
        rel_meaning = translate_biomedical_relationship(rel_type)
        rel_text = f" through {rel_meaning}"
    
    reasoning_steps = [
        f"Step 1: Identified clinical entities in question: {', '.join(entity_names)}",
        f"Step 2: Found biomedical knowledge graph path connecting {source_name} to {target_name}{rel_text}",
        f"Step 3: Path length: {selected_path.get('path_length', 'N/A')} biological relationships",
        f"Step 4: The path shows that {source_name} is biologically connected to {target_name}",
        f"Step 5: Based on this biomedical connection, we can answer the clinical question"
    ]
    
    detailed_reasoning = generate_biomedical_path_based_reasoning(selected_path, question)
    final_answer = generate_biomedical_final_answer(selected_path, question)
    
    return {
        "chain_of_thought": {
            "reasoning_steps": reasoning_steps,
            "detailed_reasoning": detailed_reasoning,
            "final_answer": final_answer
        }
    }

def format_biomedical_paths_for_display(path_results: Dict) -> str:
    """
    Format biomedical path results in a readable way.
    """
    if not path_results["entity_pairs"]:
        return "No clinical paths found between mapped entities."
    
    output_lines = []
    output_lines.append("\n" + "="*80)
    output_lines.append("BIOMEDICAL PATH SEARCH RESULTS")
    output_lines.append("="*80)
    
    output_lines.append(f"\nTotal clinical entity pairs analyzed: {len(path_results['entity_pairs'])}")
    output_lines.append(f"Total biomedical paths found: {path_results['total_paths_found']}")
    
    for pair_idx, pair in enumerate(path_results["entity_pairs"], 1):
        output_lines.append(f"\n{pair_idx}. {pair['source_entity']} ({pair['source_type']}) → {pair['target_entity']} ({pair['target_type']})")
        output_lines.append(f"   Mapped to KG: {pair['source_node']} → {pair['target_node']}")
        output_lines.append(f"   Paths found: {pair['paths_found']}")
        output_lines.append(f"   Shortest path length: {pair['shortest_path_length']} biological relationships")
        
        for path_idx, path in enumerate(pair["top_paths"][:2], 1):  # Show first 2 paths
            output_lines.append(f"\n   Path {path_idx} ({path['path_length']} hops):")
            output_lines.append(f"   {path['path_string']}")
            
            # Add biomedical context
            nodes = path.get("nodes", [])
            if nodes:
                source_labels = nodes[0].get("labels", [])
                target_labels = nodes[-1].get("labels", [])
                if source_labels or target_labels:
                    output_lines.append(f"   Context: {source_labels[0] if source_labels else 'Unknown'} → {target_labels[0] if target_labels else 'Unknown'}")
    
    return "\n".join(output_lines)

# ENHANCED MAIN PIPELINE WITH PATH PRUNING AND COT GENERATION


In [15]:
def extract_and_map_entities_with_cot(text: str, client) -> Dict:
    """
    Complete enhanced pipeline with property-aware entity mapping, path pruning, and CoT generation.
    Biomedical/fertility focused version.
    """
    print(f"\n🔍 Processing clinical question: '{text[:100]}...'" if len(text) > 100 else f"\n🔍 Processing clinical question: '{text}'")
    
    # Step 1: Extract fertility-related biomedical entities from text
    print("📝 Extracting fertility-related biomedical entities from text...")
    extracted_entities = extract_entities_from_text(text, client)
    print(f"✅ Extracted {len(extracted_entities)} clinical entities:")
    for entity in extracted_entities:
        conf = entity.get('confidence', 0.7)
        entity_type = entity.get('type', 'Unknown')
        print(f"   - {entity['name']} ({entity_type}) [Confidence: {conf:.2f}]")
    
    # Step 2: Map each entity to KG with property enhancement
    print("\n🗺️  Mapping clinical entities to biomedical knowledge graph...")
    mapped_results = []
    
    for entity in extracted_entities:
        print(f"  🔄 Mapping clinical entity: {entity['name']} ({entity.get('type', 'Unknown')})...")
        mapping_result = map_entity_to_kg_with_properties(entity, text, client)
        
        # Add stage-specific emoji with biomedical context
        stage = mapping_result.get("mapping_stage", "unknown")
        stage_emoji = {
            "property_exact": "🎯",      # Exact property match
            "property_enhanced": "📊",   # Enhanced similarity
            "property_llm": "🤖",        # LLM-assisted match
            "property_fallback": "⚠️",   # Fallback match
            "no_candidates": "❌"        # No match
        }.get(stage, "🔍")
        
        if mapping_result["mapped_node"]:
            node_info = mapping_result["mapped_node"]
            match_type = node_info.get('match_type', 'unknown')
            score = node_info.get('enhanced_score', 0)
            
            # Color code based on confidence
            if score >= 0.9:
                score_str = f"\033[92m{score:.3f}\033[0m"  # Green
            elif score >= 0.7:
                score_str = f"\033[93m{score:.3f}\033[0m"  # Yellow
            else:
                score_str = f"\033[91m{score:.3f}\033[0m"  # Red
            
            print(f"    {stage_emoji} {stage}: {entity['name']} → \033[1m{node_info['node_name']}\033[0m "
                  f"(Score: {score_str}, Match: {match_type})")
        else:
            print(f"    {stage_emoji} No biomedical mapping found for {entity['name']}")
        
        mapped_results.append(mapping_result)
    
    # Step 3: Perform biomedical path search between mapped entities
    print("\n🛤️  Searching biomedical pathways between mapped entities...")
    path_results = find_shortest_paths_between_entities(mapped_results)
    
    # Display path summary immediately
    if path_results["entity_pairs"]:
        print(f"✅ Found paths for {len(path_results['entity_pairs'])} entity pairs")
        for pair in path_results["entity_pairs"][:2]:  # Show first 2 pairs
            print(f"   • {pair['source_entity']} → {pair['target_entity']}: {pair['paths_found']} paths")
    else:
        print("❌ No biomedical pathways found between mapped entities")
    
    # Step 4: Prune paths using LLM to select the most clinically relevant one
    if path_results["all_paths"]:
        print("\n✂️  Selecting most clinically relevant path...")
        path_pruning_result = prune_paths_with_llm(text, path_results["all_paths"], mapped_results, client)
    else:
        print("\n⚠️  Skipping path pruning - no paths available")
        path_pruning_result = {"selected_path": None, "reasoning": "No paths found"}
    
    # Step 5: Generate clinical Chain-of-Thought reasoning
    print("\n🧠 Generating clinical Chain-of-Thought reasoning...")
    selected_path = path_pruning_result.get("selected_path")
    cot_result = generate_chain_of_thought(text, selected_path, mapped_results, client)
    
    # Display results in clinical format
    print("\n" + "="*80)
    print("CLINICAL ANALYSIS RESULTS")
    print("="*80)
    
    # Display entity mapping summary
    print(f"\n📊 Clinical Entity Mapping Summary:")
    successful_mappings = [m for m in mapped_results if m.get("mapped_node")]
    total_entities = len(extracted_entities)
    
    if total_entities > 0:
        success_rate = len(successful_mappings) / total_entities * 100
        print(f"   Successfully mapped: {len(successful_mappings)}/{total_entities} entities ({success_rate:.1f}%)")
        
        # Show mapping quality distribution
        high_confidence = sum(1 for m in successful_mappings 
                             if m["mapped_node"].get("enhanced_score", 0) >= 0.8)
        medium_confidence = sum(1 for m in successful_mappings 
                               if 0.6 <= m["mapped_node"].get("enhanced_score", 0) < 0.8)
        low_confidence = sum(1 for m in successful_mappings 
                            if m["mapped_node"].get("enhanced_score", 0) < 0.6)
        
        if successful_mappings:
            print(f"   Mapping confidence levels:")
            print(f"     • High (≥0.8): {high_confidence} entities")
            print(f"     • Medium (0.6-0.8): {medium_confidence} entities")
            print(f"     • Low (<0.6): {low_confidence} entities")
    
    # Display path analysis
    print(f"\n🛤️  Biomedical Pathway Analysis:")
    if selected_path:
        path_desc = selected_path.get('detailed_description', selected_path.get('path_string', 'N/A'))
        print(f"   Selected clinical pathway: {path_desc}")
        print(f"   Pathway length: {selected_path.get('path_length', 'N/A')} biological relationships")
        
        confidence = path_pruning_result.get('confidence', 0)
        confidence_color = "\033[92m" if confidence >= 0.8 else "\033[93m" if confidence >= 0.6 else "\033[91m"
        print(f"   Clinical relevance confidence: {confidence_color}{confidence:.2f}\033[0m")
        
        reasoning = path_pruning_result.get('reasoning', 'N/A')
        if len(reasoning) > 120:
            reasoning = reasoning[:120] + "..."
        print(f"   Selection rationale: {reasoning}")
        
        # Show fertility relevance if present
        fertility_keywords = ["fertility", "infertility", "reproductive", "ovarian", "uterine", "sperm", "embryo"]
        path_text = json.dumps(selected_path).lower()
        fertility_matches = [kw for kw in fertility_keywords if kw in path_text]
        if fertility_matches:
            print(f"   Fertility relevance: ✅ (mentions: {', '.join(fertility_matches[:3])})")
    else:
        print("   ⚠️  No clinically relevant pathway selected")
    
    # Display clinical reasoning
    print(f"\n🧠 Clinical Reasoning Chain:")
    if cot_result and "chain_of_thought" in cot_result:
        cot = cot_result["chain_of_thought"]
        
        # Show reasoning steps
        steps = cot.get('reasoning_steps', [])
        if steps:
            print(f"   Reasoning steps ({len(steps)} total):")
            for i, step in enumerate(steps[:3]):  # Show first 3 steps
                step_text = step.replace("Step X:", "").replace("Step X: ", "").strip()
                if step_text:
                    print(f"     {i+1}. {step_text[:80]}...")
            if len(steps) > 3:
                print(f"     ... and {len(steps) - 3} more steps")
        
        # Show final clinical answer
        final_answer = cot.get('final_answer', 'N/A')
        print(f"\n   📋 Clinical Answer:")
        print(f"   \"{final_answer}\"")
        
        # Show detailed reasoning preview
        detailed_reasoning = cot.get('detailed_reasoning', '')
        if detailed_reasoning and len(detailed_reasoning) > 50:
            print(f"\n   🔍 Reasoning preview:")
            # Get first paragraph
            first_para = detailed_reasoning.split('\n\n')[0]
            if len(first_para) > 150:
                first_para = first_para[:150] + "..."
            print(f"   {first_para}")
    else:
        print("   ⚠️  No clinical reasoning generated")
    
    # Summary of biomedical knowledge used
    print(f"\n📚 Biomedical Knowledge Used:")
    print(f"   • Knowledge graph nodes: {len(KG_NODES)} (disease/drug only)")
    print(f"   • Entity types supported: {', '.join(ENTITY_TYPES)}")
    print(f"   • Disease properties: {len(ENTITY_PROPERTY_SCHEMAS.get('disease', {}).get('properties', []))}")
    print(f"   • Drug properties: {len(ENTITY_PROPERTY_SCHEMAS.get('drug', {}).get('properties', []))}")
    
    # Prepare enhanced output with clinical context
    output = {
        "clinical_question": text,
        "extracted_entities": extracted_entities,
        "extracted_entities_count": len(extracted_entities),
        "mapped_entities": mapped_results,
        "path_search_results": path_results,
        "path_pruning_result": path_pruning_result,
        "chain_of_thought": cot_result,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "clinical_context": {
            "domain": "Fertility/Reproductive Medicine",
            "biomedical_kg_info": {
                "total_disease_drug_nodes": len(KG_NODES),
                "entity_types_focused": ENTITY_TYPES,
                "property_schemas_used": list(ENTITY_PROPERTY_SCHEMAS.keys())
            },
            "fertility_relevance": {
                "fertility_keywords_present": any(kw in text.lower() for kw in 
                                                  ["fertility", "infertility", "reproductive", "pregnancy"]),
                "extracted_fertility_entities": sum(1 for e in extracted_entities 
                                                   if any(kw in e['name'].lower() for kw in 
                                                         ["pcos", "endometriosis", "infertility", "clomiphene"]))
            }
        },
        "pipeline_config": {
            "top_k": TOP_K,
            "similarity_threshold": SIM_THRESHOLD,
            "exact_match_threshold": EXACT_MATCH_THRESHOLD,
            "property_weights": PROPERTY_WEIGHTS,
            "max_path_length": MAX_PATH_LENGTH,
            "max_paths_per_pair": MAX_PATHS_PER_PAIR,
            "top_paths_for_pruning": TOP_PATHS_FOR_PRUNING,
            "llm_model": LLM_MODEL
        }
    }
    
    return output

# PROPERTY ANALYSIS UTILITIES


In [16]:
def analyze_property_matches(results: List[Dict]):
    """Analyze property-based matching performance for biomedical entities."""
    print("\n" + "="*70)
    print("BIOMEDICAL PROPERTY MATCHING ANALYSIS")
    print("="*70)
    
    total_mapped = 0
    property_based_matches = 0
    stage_distribution = defaultdict(int)
    match_type_distribution = defaultdict(int)
    entity_type_distribution = defaultdict(lambda: defaultdict(int))
    
    for result in results:
        for mapping in result.get("mapped_entities", []):
            if mapping.get("mapped_node"):
                total_mapped += 1
                stage = mapping.get("mapping_stage", "unknown")
                stage_distribution[stage] += 1
                
                # Get entity type
                entity_type = mapping.get("query_entity", {}).get("type", "Unknown")
                entity_type_distribution[entity_type]["total"] += 1
                entity_type_distribution[entity_type][stage] += 1
                
                # Check if property matching was involved
                mapped_node = mapping.get("mapped_node", {})
                match_type = mapped_node.get("match_type", "")
                if match_type and "property" in stage.lower() or mapping.get("property_info"):
                    property_based_matches += 1
                    entity_type_distribution[entity_type]["property_based"] += 1
                
                # Track match types
                if match_type:
                    match_type_distribution[match_type] += 1
    
    print(f"Total clinical entities mapped: {total_mapped}")
    if total_mapped > 0:
        prop_percentage = property_based_matches/total_mapped*100 if total_mapped > 0 else 0
        print(f"Biomedical property-based matches: {property_based_matches} ({prop_percentage:.1f}%)")
    
    print(f"\nMapping stage distribution:")
    stage_emojis = {
        "property_exact": "🎯",
        "property_enhanced": "📊",
        "property_llm": "🤖",
        "property_fallback": "⚠️",
        "no_candidates": "❌"
    }
    for stage, count in sorted(stage_distribution.items()):
        emoji = stage_emojis.get(stage, "🔍")
        percentage = count/total_mapped*100 if total_mapped > 0 else 0
        print(f"  {emoji} {stage}: {count} ({percentage:.1f}%)")
    
    print(f"\nBiomedical match type distribution:")
    for match_type, count in sorted(match_type_distribution.items()):
        # Group similar match types
        if "exact_" in match_type:
            match_display = "🔍 Exact property matches"
        elif "contains_" in match_type:
            match_display = "📄 Contains property matches"
        elif "name" in match_type:
            match_display = "🏷️  Name matches"
        elif "property_enhanced" in match_type:
            match_display = "⚡ Property-enhanced similarity"
        else:
            match_display = f"❓ {match_type}"
        
        print(f"  {match_display}: {count}")
    
    print(f"\nEntity type distribution (disease/drug):")
    if total_mapped > 0:
        for entity_type, stats in entity_type_distribution.items():
            total = stats.get("total", 0)
            prop_based = stats.get("property_based", 0)
            prop_pct = prop_based/total*100 if total > 0 else 0
            
            entity_emoji = "🦠" if entity_type.lower() == "disease" else "💊" if entity_type.lower() == "drug" else "❓"
            print(f"  {entity_emoji} {entity_type}: {total} entities")
            print(f"     Property-based matches: {prop_based} ({prop_pct:.1f}%)")
            
            # Show stage distribution for this entity type
            for stage, count in stats.items():
                if stage not in ["total", "property_based"] and count > 0:
                    stage_emoji = stage_emojis.get(stage, "🔍")
                    print(f"     {stage_emoji} {stage}: {count}")

def generate_property_mapping_report(result: Dict):
    """Generate detailed clinical property mapping report for a single result."""
    print("\n" + "="*70)
    print("CLINICAL PROPERTY MAPPING REPORT")
    print("="*70)
    
    print(f"Clinical Question: {result.get('clinical_question', result.get('text', ''))[:120]}...")
    print(f"Extracted biomedical entities: {result.get('extracted_entities_count', 0)}")
    
    # Check fertility relevance
    fertility_context = result.get("clinical_context", {}).get("fertility_relevance", {})
    if fertility_context.get("fertility_keywords_present", False):
        print(f"Fertility relevance: ✅ (Keywords detected in question)")
    if fertility_context.get("extracted_fertility_entities", 0) > 0:
        print(f"Fertility entities extracted: {fertility_context['extracted_fertility_entities']}")
    
    # Display biomedical mapping details
    mapped_entities = result.get("mapped_entities", [])
    print(f"\nBiomedical Entity Mapping Details:")
    
    for i, mapping in enumerate(mapped_entities, 1):
        query_entity = mapping.get("query_entity", {})
        mapped_node = mapping.get("mapped_node", {})
        
        # Entity type emoji
        entity_type = query_entity.get("type", "Unknown")
        entity_emoji = "🦠" if entity_type.lower() == "disease" else "💊" if entity_type.lower() == "drug" else "❓"
        
        print(f"\n{i}. {entity_emoji} Clinical Entity: {query_entity.get('name')} ({entity_type})")
        print(f"   Extraction confidence: {query_entity.get('confidence', 0.7):.2f}")
        
        if mapped_node:
            # Score color coding
            score = mapped_node.get('enhanced_score', 0)
            if score >= 0.9:
                score_color = "\033[92m"  # Green
            elif score >= 0.7:
                score_color = "\033[93m"  # Yellow
            else:
                score_color = "\033[91m"  # Red
            
            print(f"   ✅ Mapped to KG: {mapped_node.get('node_name')}")
            print(f"   Stage: {mapped_node.get('mapping_stage')}")
            print(f"   Enhanced score: {score_color}{score:.4f}\033[0m")
            print(f"   Match type: {mapped_node.get('match_type', 'unknown')}")
            
            if "property_match_type" in mapped_node:
                prop_match = mapped_node['property_match_type']
                if "exact_primary_key" in prop_match:
                    print(f"   🎯 Property match: {prop_match}")
                elif "exact_display_property" in prop_match:
                    print(f"   📊 Property match: {prop_match}")
                elif "contains" in prop_match:
                    print(f"   🔍 Property match: {prop_match}")
            
            # Show matched KG node labels
            kg_labels = mapped_node.get('labels', [])
            if kg_labels:
                print(f"   KG labels: {', '.join(kg_labels)}")
            
            # Show fertility relevance if available
            if query_entity.get('name', '').lower() in ['pcos', 'endometriosis', 'infertility', 'clomiphene', 'letrozole']:
                print(f"   🏥 Fertility relevance: High")
        else:
            print(f"   ❌ Not mapped to knowledge graph")
            print(f"   Stage: {mapping.get('mapping_stage', 'unknown')}")
            print(f"   Reason: {mapping.get('reason', 'No match found')}")
        
        # Detailed property information
        property_info = mapping.get("property_info", {})
        if property_info:
            print(f"\n   🔬 Biomedical Property Analysis:")
            
            # Show match type
            match_type = property_info.get("match_type", "")
            if match_type:
                print(f"     Match type: {match_type}")
            
            # Show matched properties
            matched_props = property_info.get("matched_properties", [])
            if matched_props:
                print(f"     Matched biomedical properties: {', '.join(matched_props[:5])}")
                if len(matched_props) > 5:
                    print(f"     ... and {len(matched_props) - 5} more")
            
            # Show specific property matches
            matched_property = property_info.get("matched_property")
            if matched_property:
                print(f"     Key matched property: {matched_property}")
                prop_value = property_info.get("property_value", "")
                if prop_value:
                    if len(str(prop_value)) > 80:
                        prop_value = str(prop_value)[:80] + "..."
                    print(f"     Property value: {prop_value}")
            
            # Show score breakdown
            score_breakdown = property_info.get("score_breakdown", {})
            if score_breakdown:
                print(f"     Score breakdown:")
                for key, value in score_breakdown.items():
                    if isinstance(value, float):
                        print(f"       • {key}: {value:.3f}")
                    else:
                        print(f"       • {key}: {value}")
    
    # Path analysis section
    path_pruning = result.get("path_pruning_result", {})
    if path_pruning.get("selected_path"):
        print(f"\n🛤️  Selected Biomedical Pathway:")
        selected_path = path_pruning["selected_path"]
        print(f"   Pathway: {selected_path.get('detailed_description', selected_path.get('path_string', ''))}")
        print(f"   Length: {selected_path.get('path_length', 'N/A')} biological relationships")
        print(f"   Selection confidence: {path_pruning.get('confidence', 0):.2f}")
        
        # Show entities in path
        nodes = selected_path.get("nodes", [])
        if nodes:
            print(f"   Entities in pathway:")
            for j, node in enumerate(nodes[:3]):  # Show first 3
                node_labels = node.get("labels", [])
                label_str = node_labels[0] if node_labels else "Unknown"
                print(f"     {j+1}. {node.get('name', 'Unknown')} ({label_str})")
            if len(nodes) > 3:
                print(f"     ... and {len(nodes) - 3} more")
    
    # Clinical reasoning summary
    cot_result = result.get("chain_of_thought", {})
    if cot_result and "chain_of_thought" in cot_result:
        cot = cot_result["chain_of_thought"]
        print(f"\n🧠 Clinical Reasoning Summary:")
        print(f"   Final clinical answer: {cot.get('final_answer', 'N/A')}")
        
        steps = cot.get('reasoning_steps', [])
        if steps and len(steps) > 0:
            print(f"   First reasoning step: {steps[0][:80]}...")

# ENHANCED DEMO WITH PATH PRUNING AND COT GENERATION


In [17]:
def run_complete_pipeline_demo(client):
    """Run complete clinical pipeline with entity extraction, mapping, path pruning, and CoT generation."""
    
    # Test cases with clinical questions that benefit from path analysis
    test_cases = [
        "Is Clomiphene citrate effective for PCOS?"
    ]
    
    print("\n" + "="*80)
    print("CLINICAL BIOMEDICAL PIPELINE WITH PATH PRUNING & COT GENERATION")
    print("="*80)
    print(f"🔬 Using {len(KG_NODES)} disease/drug KG nodes with biomedical property-aware matching\n")
    
    all_results = []
    
    for i, text in enumerate(test_cases, 1):
        print(f"\n📋 CLINICAL TEST CASE {i}:")
        print("-" * 80)
        
        result = extract_and_map_entities_with_cot(text, client)
        all_results.append(result)
        
        # Quick clinical summary
        print(f"\n📊 Clinical Summary for Test {i}:")
        print(f"   Clinical Question: {text}")
        print(f"   Biomedical entities extracted: {result['extracted_entities_count']}")
        
        # Show specific entities if extracted
        entities = result.get('extracted_entities', [])
        if entities:
            entity_types = []
            for entity in entities[:3]:  # Show first 3
                entity_type = entity.get('type', 'Unknown')
                entity_emoji = "🦠" if entity_type.lower() == "disease" else "💊" if entity_type.lower() == "drug" else "❓"
                entity_types.append(f"{entity_emoji} {entity['name']}")
            if len(entities) > 3:
                entity_types.append(f"... +{len(entities)-3} more")
            print(f"   Extracted entities: {', '.join(entity_types)}")
        
        print(f"   Biomedical pathways found: {result['path_search_results']['total_paths_found']}")
        
        # Show selected pathway
        if result.get('path_pruning_result', {}).get('selected_path'):
            selected_path = result['path_pruning_result']['selected_path']
            path_desc = selected_path.get('detailed_description', selected_path.get('path_string', 'N/A'))
            if len(path_desc) > 120:
                path_desc = path_desc[:120] + "..."
            print(f"   Selected clinical pathway: {path_desc}")
            confidence = result['path_pruning_result'].get('confidence', 0)
            
            # Color code confidence
            if confidence >= 0.8:
                conf_color = "\033[92m"  # Green
            elif confidence >= 0.6:
                conf_color = "\033[93m"  # Yellow
            else:
                conf_color = "\033[91m"  # Red
            print(f"   Pathway relevance confidence: {conf_color}{confidence:.2f}\033[0m")
        
        # Show final clinical answer
        cot_data = result.get('chain_of_thought', {}).get('chain_of_thought', {})
        if cot_data and cot_data.get('final_answer'):
            answer = cot_data['final_answer']
            print(f"   📋 Clinical Answer: \"{answer}\"")
            
            # Show reasoning step count
            steps = cot_data.get('reasoning_steps', [])
            if steps:
                print(f"   Reasoning steps: {len(steps)}")
        
        # Show fertility relevance if present
        fertility_ctx = result.get('clinical_context', {}).get('fertility_relevance', {})
        if fertility_ctx.get('fertility_keywords_present', False):
            print(f"   🏥 Fertility context detected")
    
    # Overall summary
    print(f"\n" + "="*80)
    print("CLINICAL PIPELINE DEMO COMPLETE")
    print("="*80)
    
    total_questions = len(all_results)
    total_entities = sum(r.get('extracted_entities_count', 0) for r in all_results)
    total_paths = sum(r.get('path_search_results', {}).get('total_paths_found', 0) for r in all_results)
    questions_with_paths = sum(1 for r in all_results 
                              if r.get('path_search_results', {}).get('total_paths_found', 0) > 0)
    questions_with_answers = sum(1 for r in all_results 
                                if r.get('chain_of_thought', {}).get('chain_of_thought', {}).get('final_answer'))
    
    print(f"\n📈 Overall Clinical Performance:")
    print(f"   Clinical questions analyzed: {total_questions}")
    print(f"   Total biomedical entities extracted: {total_entities}")
    print(f"   Total biomedical pathways found: {total_paths}")
    print(f"   Questions with pathways: {questions_with_paths}/{total_questions} ({questions_with_paths/total_questions*100:.1f}%)")
    print(f"   Questions answered: {questions_with_answers}/{total_questions} ({questions_with_answers/total_questions*100:.1f}%)")
    
    # Analyze entity type distribution
    disease_count = 0
    drug_count = 0
    for result in all_results:
        for entity in result.get('extracted_entities', []):
            entity_type = entity.get('type', '').lower()
            if entity_type == 'disease':
                disease_count += 1
            elif entity_type == 'drug':
                drug_count += 1
    
    if total_entities > 0:
        print(f"\n🧬 Biomedical Entity Distribution:")
        print(f"   🦠 Diseases: {disease_count} ({disease_count/total_entities*100:.1f}%)")
        print(f"   💊 Drugs: {drug_count} ({drug_count/total_entities*100:.1f}%)")
    
    # Run property matching analysis
    if all_results:
        print(f"\n" + "="*80)
        analyze_property_matches(all_results)
    
    return all_results

# MAIN EXECUTION


In [18]:
# Initialize embedding model
try:
    print("🧬 Loading biomedical sentence transformer model...")
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    print("✅ Embedding model loaded")
except Exception as e:
    print(f"❌ Error loading embedding model: {e}")
    exit(1)
    
    # Generate embeddings for all KG nodes
print("\n🧬 Generating biomedical embeddings for knowledge graph nodes...")
KG_EMBEDDINGS = embedder.encode(KG_TEXTS)
print(f"✅ Generated embeddings for {len(KG_EMBEDDINGS)} disease/drug nodes")
    

🧬 Loading biomedical sentence transformer model...
✅ Embedding model loaded

🧬 Generating biomedical embeddings for knowledge graph nodes...
✅ Generated embeddings for 25037 disease/drug nodes


In [ ]:
if __name__ == "__main__":
    # Initialize Groq client for LLM
    try:
        from groq import Groq
        client = Groq(api_key="groq_api")
        print("✅ Groq client initialized for clinical reasoning")
    except ImportError:
        print("⚠️ Groq client not available. Using mock client for testing.")
        class MockClient:
            class chat:
                class completions:
                    @staticmethod
                    def create(**kwargs):
                        class MockResponse:
                            class Choice:
                                class Message:
                                    content = '{"Entity": [{"id": "1", "type": "disease", "name": "Polycystic ovary syndrome", "confidence": 0.9}, {"id": "2", "type": "drug", "name": "Metformin", "confidence": 0.8}]}'
                                message = Message()
                            choices = [Choice()]
                        return MockResponse()
        client = MockClient()
    
    
    # Run complete clinical pipeline demo
    print("\n" + "="*80)
    print("🚀 STARTING COMPLETE CLINICAL PIPELINE DEMO")
    print("="*80)
    
    results = run_complete_pipeline_demo(client)
    
    # Save detailed clinical results
    if results:
        output_file = "clinical_pipeline_results.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results[0], f, indent=2, default=str, ensure_ascii=False)
        print(f"\n💾 Clinical results saved to '{output_file}'")
        
        # Display sample clinical CoT from first result
        if results[0].get('chain_of_thought', {}).get('chain_of_thought'):
            cot = results[0]['chain_of_thought']['chain_of_thought']
            print("\n" + "="*80)
            print("SAMPLE CLINICAL CHAIN-OF-THOUGHT OUTPUT")
            print("="*80)
            
            # Display clinical question
            clinical_question = results[0].get('clinical_question', results[0].get('text', 'Unknown'))
            print(f"\n📋 Clinical Question: {clinical_question}")
            
            # Display final clinical answer
            final_answer = cot.get('final_answer', 'N/A')
            print(f"\n🏥 Final Clinical Answer: {final_answer}")
            
            # Display reasoning steps
            steps = cot.get('reasoning_steps', [])
            if steps:
                print(f"\n🧠 Clinical Reasoning Steps ({len(steps)} total):")
                for i, step in enumerate(steps):
                    # Clean up step text
                    step_text = step.replace("Step X:", "").replace("Step X: ", "").strip()
                    if step_text:
                        print(f"\n{i+1}. {step_text}")
            
            # Display detailed reasoning preview
            detailed = cot.get('detailed_reasoning', '')
            if detailed:
                print(f"\n🔍 Detailed Clinical Reasoning Preview:")
                paragraphs = detailed.split('\n\n')
                for para in paragraphs[:2]:  # Show first 2 paragraphs
                    if para.strip():
                        if len(para) > 120:
                            para = para[:120] + "..."
                        print(f"   {para}")
                if len(paragraphs) > 2:
                    print(f"   ... and {len(paragraphs) - 2} more paragraphs")
        
        # Generate comprehensive clinical report
        print(f"\n" + "="*80)
        print("CLINICAL PERFORMANCE REPORT")
        print("="*80)
        
        # Calculate statistics
        total_entities_mapped = 0
        high_confidence_mappings = 0
        fertility_questions = 0
        
        for result in results:
            # Count mapped entities
            for mapping in result.get('mapped_entities', []):
                if mapping.get('mapped_node'):
                    total_entities_mapped += 1
                    if mapping['mapped_node'].get('enhanced_score', 0) >= 0.8:
                        high_confidence_mappings += 1
            
            # Count fertility questions
            fertility_ctx = result.get('clinical_context', {}).get('fertility_relevance', {})
            if fertility_ctx.get('fertility_keywords_present', False):
                fertility_questions += 1
        
        print(f"\n📊 Clinical Pipeline Statistics:")
        print(f"   Total test cases: {len(results)}")
        print(f"   Fertility-related questions: {fertility_questions}/{len(results)} ({fertility_questions/len(results)*100:.1f}%)")
        print(f"   Total biomedical entities mapped: {total_entities_mapped}")
        
        if total_entities_mapped > 0:
            high_conf_pct = high_confidence_mappings/total_entities_mapped*100
            print(f"   High-confidence mappings (≥0.8): {high_confidence_mappings}/{total_entities_mapped} ({high_conf_pct:.1f}%)")
        
        # Show KG statistics
        print(f"\n📚 Biomedical Knowledge Graph Statistics:")
        print(f"   Disease/drug nodes loaded: {len(KG_NODES)}")
        
        # Count disease vs drug nodes
        disease_nodes = sum(1 for node in KG_NODES if node.get('entity_type') == 'disease')
        drug_nodes = sum(1 for node in KG_NODES if node.get('entity_type') == 'drug')
        print(f"   Disease nodes: {disease_nodes} ({disease_nodes/len(KG_NODES)*100:.1f}%)")
        print(f"   Drug nodes: {drug_nodes} ({drug_nodes/len(KG_NODES)*100:.1f}%)")
        
        # Property schema information
        print(f"\n🔬 Biomedical Property Schemas:")
        for entity_type, schema in ENTITY_PROPERTY_SCHEMAS.items():
            prop_count = len(schema.get('properties', []))
            primary_keys = schema.get('primary_keys', [])
            print(f"   {entity_type.upper()}: {prop_count} properties, Primary keys: {', '.join(primary_keys)}")
    
    print(f"\n🎉 Clinical pipeline execution complete!")
    print(f"   Date: {time.strftime('%Y-%m-%d %H:%M:%S')}")

✅ Groq client initialized for clinical reasoning

🚀 STARTING COMPLETE CLINICAL PIPELINE DEMO

CLINICAL BIOMEDICAL PIPELINE WITH PATH PRUNING & COT GENERATION
🔬 Using 25037 disease/drug KG nodes with biomedical property-aware matching


📋 CLINICAL TEST CASE 1:
--------------------------------------------------------------------------------

🔍 Processing clinical question: 'Is Clomiphene citrate effective for PCOS?'
📝 Extracting fertility-related biomedical entities from text...
✅ Extracted 2 clinical entities:
   - PCOS (disease) [Confidence: 0.90]
   - Clomiphene citrate (drug) [Confidence: 0.95]

🗺️  Mapping clinical entities to biomedical knowledge graph...
  🔄 Mapping clinical entity: PCOS (disease)...
    🔍 unknown: PCOS → polycystic ovary syndrome (Score: 0.388, Match: property_enhanced)
  🔄 Mapping clinical entity: Clomiphene citrate (drug)...
    🔍 unknown: Clomiphene citrate → Clomifene (Score: 0.236, Match: text_similarity_only)

🛤️  Searching biomedical pathways between mappe